### 00meta data files
- Data availabilty
- Gap composition
- machine_details
- measurement details

### 1.) Data availability File

In [34]:
from pathlib import Path

paths = [Path(r"..\dataset_clean"), Path("../dataset_clean_validation")]
for p in paths:
    count = sum(1 for _ in p.rglob("*") if _.is_file())
    print(f"{p.resolve()}: {count} files")


D:\EnergyDataset\dataset_clean: 14155 files
D:\EnergyDataset\dataset_clean_validation: 30710 files


In [13]:
# Data availability File
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path

# --- SETTINGS ---
base_dir = Path("../dataset_clean_validation")  # adjust if needed

# --- COLLECT DATA ---
records = []
for file in base_dir.rglob("*_missing.csv"):
    try:
        df = pd.read_csv(file)
    except Exception as e:
        print(f"Skipping {file}: {e}")
        continue
    
    if "n_total" not in df.columns or "n_nans" not in df.columns:
        continue  # skip irrelevant files

    # Compute availability
    n_total = df.loc[0, "n_total"]
    n_nans = df.loc[0, "n_nans"]
    availability = 1 - n_nans / n_total if n_total > 0 else 0

    # Parse machine, measurement, year from path
    # Example: validation_results\EPI_ChipPress\Freq\2018_Freq_missing.csv
    try:
        machine = file.parts[1]      # e.g. "EPI_ChipPress"
        measurement = file.parts[2]  # e.g. "Freq"
        year = int(file.stem.split("_")[0])  # e.g. "2018"
    except Exception:
        continue

    records.append([machine, measurement, year, availability])

df_all = pd.DataFrame(records, columns=["machine", "measurement", "year", "availability"])
#df_all.to_csv("dataset_clean/00meta_data/02data_availability.csv")

,machine,measurement,year,availability
0,dataset_clean_validation,EPI_ChipPress,2018,0.894253
1,dataset_clean_validation,EPI_ChipPress,2019,0.680976
2,dataset_clean_validation,EPI_ChipPress,2020,0.711715
3,dataset_clean_validation,EPI_ChipPress,2021,0.720484
4,dataset_clean_validation,EPI_ChipPress,2022,0.793986
...,...,...,...,...
15350,dataset_clean_validation,TEC_MV2400R,2024,0.906009
15351,dataset_clean_validation,TEC_MV2400R,2024,0.951372
15352,dataset_clean_validation,TEC_MV2400R,2024,0.905821
15353,dataset_clean_validation,TEC_MV2400R,2024,0.951371


### 2.) Gap composiition

In [11]:
#Gap composition file
# -*- coding: utf-8 -*-
"""
Gap composition per machine (length distribution + start/middle/end position)
Collapsed bins: <60min, 1h–1d, 1d–1w, >1w
"""

from pathlib import Path
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Patch

# ----------------------------- Paths ---------------------------------
BASE_VALIDATION = Path("../dataset_clean_validation/")
META_DIR = Path("../dataset_clean") / "00meta_data"
#META_DIR.mkdir(parents=True, exist_ok=True)

FILTER_FILE = META_DIR / "02data_availability.csv"
OUT_CSV = META_DIR / "03gap_composition_by_machine.csv"

# --------------------------- Source bins ------------------------------
# Raw columns (from your _missing.csv files)
RAW_BINS = [
    ("nan_1_step_steps",   "1 step"),
    ("nan_5s_30s_steps",   "5s–30s"),
    ("nan_30s_1m_steps",   "30s–1m"),
    ("nan_1m_15m_steps",   "1m–15m"),
    ("nan_15m_1h_steps",   "15m–1h"),
    ("nan_1h_24h_steps",   "1h–24h"),
    ("nan_1d_7d_steps",    "1d–7d"),
    ("nan_1w_1mo_steps",   "1w–1mo"),
    ("nan_1mo_inf_steps",  "≥1mo"),
]
POS_COLS = [("n_nans_start","Start"), ("n_nans_middle","Middle"), ("n_nans_end","End")]

# Collapsed bins you asked for (as sums of RAW_BINS)
COLLAPSE = {
    "<60min":  ["1 step", "5s–30s", "30s–1m", "1m–15m", "15m–1h"],
    "1h–1d":   ["1h–24h"],
    "1d–1w":   ["1d–7d"],
    ">1w":     ["1w–1mo", "≥1mo"],
}

# --------------------------- Utilities -------------------------------
def parse_triplet_from_path(fp: Path):
    try:
        machine = fp.parts[2]          # validation_results/<machine>/...
        measurement = fp.parts[3]
        m = re.search(r"(\d{4})_", fp.name)
        year = int(m.group(1)) if m else None
        return machine, measurement, year
    except Exception:
        return None, None, None

# --------------------------- Build filter ----------------------------
allowed = None
if FILTER_FILE.exists():
    f = pd.read_csv(FILTER_FILE, dtype={"machine":str, "measurement":str, "year":int})
    allowed = set(zip(f["machine"].str.strip(), f["measurement"].str.strip(), f["year"].astype(int)))
    order_by_avail = (
        f.groupby("machine")["availability"].median()
         .sort_values(ascending=True).index.tolist()
    )
else:
    order_by_avail = None

# ------------------------ Aggregate per machine ----------------------
agg = {}
files = list(BASE_VALIDATION.rglob("*_missing.csv"))

for fp in files:
    machine, measurement, year = parse_triplet_from_path(fp)
    if not machine or year is None:
        continue
    if allowed is not None and (machine, measurement, year) not in allowed:
        continue

    try:
        row = pd.read_csv(fp, nrows=1).iloc[0]
    except Exception:
        continue

    n_nans = float(row.get("n_nans", 0) or 0)
    miss_sec = float(row.get("missing_gap_total_sec", 0) or 0)
    sec_per_step = (miss_sec / n_nans) if n_nans > 0 else 5.0

    if machine not in agg:
        agg[machine] = {"files": 0, "missing_total_sec": 0.0}
        # initialize raw bins + positions
        for _, lbl in RAW_BINS:      agg[machine][lbl] = 0.0
        for _, lbl in POS_COLS:      agg[machine][lbl] = 0.0

    agg[machine]["files"] += 1
    agg[machine]["missing_total_sec"] += miss_sec

    # accumulate raw bin seconds (if these were steps, we’ll rescale later)
    for col, lbl in RAW_BINS:
        agg[machine][lbl] += float(row.get(col, 0) or 0)

    # positions are counts of missing steps -> convert to seconds
    for col, lbl in POS_COLS:
        steps = float(row.get(col, 0) or 0)
        agg[machine][lbl] += steps * sec_per_step

# Build DataFrame
if not agg:
    raise RuntimeError("No _missing.csv files were found/used. Check paths and FILTER_FILE.")

df = pd.DataFrame.from_dict(agg, orient="index").reset_index().rename(columns={"index":"machine"})
raw_labels = [lbl for _, lbl in RAW_BINS]

# If raw bin sum is far from total seconds, rescale (handles step-vs-sec ambiguity)
bins_sum = df[raw_labels].sum(axis=1)
scale = np.where((bins_sum > 0) & (np.abs(bins_sum - df["missing_total_sec"]) > 0.2*df["missing_total_sec"]),
                 df["missing_total_sec"] / bins_sum, 1.0)
for lbl in raw_labels:
    df[lbl] = df[lbl] * scale

# Collapse to requested 4 bins (seconds)
for k, src in COLLAPSE.items():
    df[k] = df[src].sum(axis=1)

# Convert to percentages (composition)
collapsed_labels = list(COLLAPSE.keys())
for lbl in collapsed_labels:
    df[lbl] = 100.0 * df[lbl] / df["missing_total_sec"].replace(0, np.nan)
for lbl in ["Start","Middle","End"]:
    df[lbl] = 100.0 * df[lbl] / df["missing_total_sec"].replace(0, np.nan)
df[collapsed_labels + ["Start","Middle","End"]] = df[collapsed_labels + ["Start","Middle","End"]].fillna(0.0)

# Save tidy CSV
out_cols = ["machine","files","missing_total_sec"] + collapsed_labels + ["Start","Middle","End"]
#df[out_cols].to_csv(OUT_CSV, index=False)
print(f"[saved] {OUT_CSV}")

[saved] ..\dataset_clean\00meta_data\gap_composition_by_machine.csv


### 3.) Machine Details

In [14]:
"""Build Machine Metadata dataset_clean\00meta_data\00machine_details.json"""

from pathlib import Path
from calendar import isleap
import re, json, csv
from functools import lru_cache

def build_machine_metadata(
    dataset_root="../dataset_clean",
    validation_root="../dataset_clean_validation",
    save=False,
    filename="00machine_details2.json",
):
    """
    Prints and returns {Machine: Info, ...} with:
      - long_desc from provided descriptions
      - measurements, num_measurements
      - phases: 3 for EPI_PV or if >100 measurements, else 1
      - years from FIRST measurement folder; num_years
      - time_steps: sum over years at 5 s resolution (int)
      - availability: step-weighted across years (float, unrounded)
      - data_points: int(round(time_steps * availability)) or None
    """
    root, vroot = Path(dataset_root), Path(validation_root)
    if not root.exists():
        raise FileNotFoundError(f"Not found: {root}")

    machines = sorted(
        p.name for p in root.iterdir()
        if p.is_dir() and p.name not in {"00meta_data"} and not p.name.startswith(".")
    )

    base_long = {
        # EPI
        "EPI_PickAndPlace": "The ESSEMTEC Paraquda PARA-4 is an automated SMT assembly system for electronic component placement (url = https://www.smd-tec.be/product-page/multi-functional-pick-place).",
        "EPI_SolderOven": "The SMT XXS N2 reflow oven (url = https://www.sztech-smt.com/product/nitrogen-reflow-oven/).",
        "EPI_WashingMachine": "The Miele IR 6002 is an industrial washer (url = https://www.medenwald.eu/miele-ir-6002/) ",
        "EPI_ScreenPrinter": "The EKRA XH STS is an automated stencil printing system (url = https://pass-systemsupply.com/products/screen-printing-machine/)",
        "EPI_HighTempOven": "The ATV PEO 604 quartz furnace enables high-temperature processes (url = https://www.atv-tech.com/products/peo-604/)",
        "EPI_VacuumSoldering": "The ATV SRO 700 vacuum soldering system (url = https://www.atv-tech.com/products/sro-700/)",
        "EPI_ChipSaw": "The Disco DAD 3240 precision dicing saw (url = https://revelationmachinery.com/product/disco-dad3240-dicing-saw-2018/).",
        "EPI_ChipPress": "The HML MP 50 1 VK chip press (url = https://hml-hm.com/en/system-portfolio-2/multilayer-press-systems/)",
        "EPI_TotalLoad": "Centralized power measurement point capturing the total facility load",
        "EPI_PumpStation1": "Infrastructure subsystem for liquid or gas transport in production support processes.",
        "EPI_PumpStation2": "Secondary pumping infrastructure with similar functionality to Pump Station 1.",
        "EPI_PV": "Rooftop solar with 10.72 kWp nominal capacity and with Sunny Boy 3.0 1AV-41 and Sunny Tripower 8.0 inverters (url = https://www.europe-solarstore.com/sma-sunny-boy-3-0-1av-41.html).",
        # TEC
        "TEC_E30D2": "he WEILER E30D2 is a precision lathe for large work pieces (url = https://www.weiler.de/produkte/drehmaschinen/zyklengesteuert/e-30/)",
        "TEC_48S": "The Lathe 48 S is a medium-duty turning machine (url = https://althaus-maschinen.de/maschinen/metallbearbeitung/drehmaschinen/drehmaschine-vdf-boehringer-goeppingen-48s-2/)",
        "TEC_CTX800TC": "The CTX 800 TC is a Lathe Milling machine (url = https://de.dmgmori.com/produkte/maschinen/drehen/drehfraesen/ctx-tc)",
        "TEC_DNG50evo": "The DNG50evo is a CNC milling system (url = https://de.dmgmori.com/produkte/maschinen/fraesen/5-achs-fraesen/evo)",
        "TEC_MV2400R": "The MV2400R is a Wire EDM Machine (url = https://www.mitsubishielectric-edm.de/produkte-und-loesungen/drahterosion/mv-r-serie-power-for-precision/)",
        "TEC_Chiron800": "The Chiron 800 is a 5-axis CNC mill (url = https://chiron-group.com/de/kompetenzen/maschinenpool/mill-800-five-axis-baseline-382-18)",
        "TEC_DMF3008": "The DMF 300/8 is a dynamic 5-axis milling machine (url = https://de.dmgmori.com/produkte/maschinen/fraesen/5-achs-fraesen/dmf/dmf-300-8-fd)",
        "TEC_DMU125MB": "The DMU 125 MB is a 5-axis milling machine (url = https://de.dmgmori.com/produkte/maschinen/fraesen/5-achs-fraesen/monoblock/dmu-125-fd-monoblock)",
        "TEC_E110": "The E110 is a heavy-duty lathe (url = https://www.weiler.de/en/cycle-controlled/e-110/)",
        "TEC_CFST161": "The CF ST 161 is an industrial chiller (url = https://www.hosbv.com/en/product/6731/waterchillers/BlueboxWestern-ALFA-ST-161.html)",
        "TEC_JWA24": "The JWA 24 liquid cooling unit (url = https://ktk.it/de/produkte/junior-line-2/air-cooled-liquid-chillers-and-heat-pumps/jwa-2440-sikpa/)",
    }

    def fallback_desc(name: str) -> str:
        site, model = name.split("_", 1) if "_" in name else ("", name)
        return f"Industrial machine '{model}' at {site} site."

    # Availability index: machine -> year -> list[availability]
    avail_idx = {}
    avail_path = root / "00meta_data" / "02data_availability.csv"
    if avail_path.exists():
        with avail_path.open("r", encoding="utf-8") as f:
            for row in csv.DictReader(f):
                try:
                    mach = row["machine"]; year = int(row["year"]); av = float(row["availability"])
                except Exception:
                    continue
                avail_idx.setdefault(mach, {}).setdefault(year, []).append(av)

    year_re = re.compile(r"^(\d{4})_.*\.csv\.xz$", re.IGNORECASE)

    def years_in_first_measurement(machine: str):
        mroot = root / machine
        folders = sorted(
            d for d in (mroot.iterdir() if mroot.exists() else [])
            if d.is_dir() and d.name not in {"000_metadata", "00meta_data"} and not d.name.startswith(".")
        )
        if not folders:
            return []
        first = folders[0]
        years = {int(m.group(1)) for fp in first.glob("*.csv.xz") if (m := year_re.match(fp.name))}
        return sorted(years)

    @lru_cache(maxsize=None)
    def steps_per_year(y: int) -> int:
        return (366 if isleap(y) else 365) * 24 * 60 * 60 // 5  # 5 s resolution

    out = {}

    for m in machines:
        # measurements from validation root
        meas_dir = vroot / m
        measurements = sorted(
            d.name for d in (meas_dir.iterdir() if meas_dir.exists() else [])
            if (meas_dir / d.name).is_dir() and d.name not in {"000_metadata", "00meta_data"} and not d.name.startswith(".")
        )
        num_meas = len(measurements)

        phases = 3 if (m == "EPI_PV" or num_meas > 100) else 1

        years = years_in_first_measurement(m)
        time_steps = int(sum(steps_per_year(y) for y in years))  # int by construction

        # step-weighted availability across years (unrounded float)
        numer = 0.0
        denom = 0
        for y in years:
            avs = avail_idx.get(m, {}).get(y)
            if avs:
                numer += (sum(avs) / len(avs)) * steps_per_year(y)
                denom += steps_per_year(y)
        availability = None if denom == 0 else (numer / denom)

        # INT data_points as requested
        data_points = None if availability is None else int(round(time_steps * availability))

        out[m] = {
            "long_desc": base_long.get(m, fallback_desc(m)),
            "measurements": measurements,
            "num_measurements": num_meas,
            "phases": phases,
            "num_years": len(years),
            "years": years,
            "time_steps": time_steps,
            "availability": availability,   # unrounded float
            "data_points": data_points,     # int
        }

    # Print ALL results (entire mapping)
    #print(json.dumps(out, ensure_ascii=False, indent=2))

    if save:
        out_dir = root / "00meta_data" 
        out_dir.mkdir(parents=True, exist_ok=True)
        out_path = out_dir / filename
        out_path.write_text(json.dumps(out, ensure_ascii=False, indent=2), encoding="utf-8")
        print(f"[✓] Saved → {out_path}")

    return out

# Example:
build_machine_metadata(save=True)

[✓] Saved → ..\dataset_clean\00meta_data\00machine_details2.json


{'01exogenous_data': {'long_desc': "Industrial machine 'data' at 01exogenous site.",
  'measurements': [],
  'num_measurements': 0,
  'phases': 1,
  'num_years': 0,
  'years': [],
  'time_steps': 0,
  'availability': None,
  'data_points': None},
 'EPI_ChipPress': {'long_desc': 'The HML MP 50 1 VK chip press (url = https://hml-hm.com/en/system-portfolio-2/multilayer-press-systems/)',
  'measurements': ['Freq',
   'I1',
   'I1_h11',
   'I1_h13',
   'I1_h15',
   'I1_h17',
   'I1_h19',
   'I1_h21',
   'I1_h23',
   'I1_h25',
   'I1_h27',
   'I1_h29',
   'I1_h3',
   'I1_h31',
   'I1_h5',
   'I1_h7',
   'I1_h9',
   'I2',
   'I2_h11',
   'I2_h13',
   'I2_h15',
   'I2_h17',
   'I2_h19',
   'I2_h21',
   'I2_h23',
   'I2_h25',
   'I2_h27',
   'I2_h29',
   'I2_h3',
   'I2_h31',
   'I2_h5',
   'I2_h7',
   'I2_h9',
   'I3',
   'I3_h11',
   'I3_h13',
   'I3_h15',
   'I3_h17',
   'I3_h19',
   'I3_h21',
   'I3_h23',
   'I3_h25',
   'I3_h27',
   'I3_h29',
   'I3_h3',
   'I3_h31',
   'I3_h5',
   'I3_h7'

### 4.) Measurement Details

In [23]:
EPI_CHANNELS_1_MEASUREMENT_DICT  = {
    #0: {'unit': 'h', 'short_desc': 'OpHours', 'long_desc': 'Operating hours counted'},
    1: {'unit': 'V', 'short_desc': 'U1', 'long_desc': 'Effective value of the Phase voltage of phase 1 to neutral (U1)'},
    2: {'unit': 'Hz', 'short_desc': 'Freq', 'long_desc': 'Network frequency'},
    3: {'unit': 'A', 'short_desc': 'I1', 'long_desc': 'Effective value of the Current in phase 1 (I1)'},
    4: {'unit': 'W', 'short_desc': 'P_total', 'long_desc': 'Total active power (DIN EN 61557-12 >0: Demand)'},
    5: {'unit': 'var', 'short_desc': 'Q_total', 'long_desc': 'Total reactive power (vectorial sum, >0: Demand)'},
    6: {'unit': 'VA', 'short_desc': 'S_total', 'long_desc': 'Total apparent power (vectorial sum)'},
    7: {'unit': '-', 'short_desc': 'PF_total', 'long_desc': 'Power factor from vectorial power (>0: Inductive, <0: Capacitive)'},
    #8: {'unit': 'kWh', 'short_desc': 'Ea_pos', 'long_desc': 'Positive active energy'},
    #9: {'unit': 'kvarh', 'short_desc': 'Er_pos', 'long_desc': 'Positive reactive energy'},
    #10: {'unit': 'kVAh', 'short_desc': 'Es_total', 'long_desc': 'Total apparent energy (kVAh)'},
    #11: {'unit': 'kWh', 'short_desc': 'Ea_neg', 'long_desc': 'Negative active energy'},
    #12: {'unit': 'kvarh', 'short_desc': 'Er_neg', 'long_desc': 'Negative reactive energy'},

    13: {'unit': '%', 'short_desc': 'THD_U1', 'long_desc': 'Total Harmonic Distortion of phase voltage U1'},
    14: {'unit': '%', 'short_desc': 'THD_I1', 'long_desc': 'Total Harmonic Distortion of current I1'},
    15: {'unit': '%', 'short_desc': 'THD_IN', 'long_desc': 'Total Harmonic Distortion of neutral current IN'},
    
    #16: {'unit': 'R', 'short_desc': 'Max_Harmonic', 'long_desc': 'Highest detectable harmonic (63)'},

    17: {'unit': '%', 'short_desc': 'I1_h3', 'long_desc': '3th harmonic of current I1 relative to fundamental'},
    18: {'unit': '%', 'short_desc': 'IN_h3', 'long_desc': '3th harmonic of neutral current IN relative to fundamental'},
    
    19: {'unit': '%', 'short_desc': 'I1_h5', 'long_desc': '5th harmonic of current I1 relative to fundamental'},
    20: {'unit': '%', 'short_desc': 'IN_h5', 'long_desc': '5th harmonic of neutral current IN relative to fundamental'},
    
    21: {'unit': '%', 'short_desc': 'I1_h7', 'long_desc': '7th harmonic of current I1 relative to fundamental'},
    22: {'unit': '%', 'short_desc': 'IN_h7', 'long_desc': '7th harmonic of neutral current IN relative to fundamental'},
    
    23: {'unit': '%', 'short_desc': 'I1_h9', 'long_desc': '9th harmonic of current I1 relative to fundamental'},
    24: {'unit': '%', 'short_desc': 'IN_h9', 'long_desc': '9th harmonic of neutral current IN relative to fundamental'},
    
    25: {'unit': '%', 'short_desc': 'I1_h11', 'long_desc': '11th harmonic of current I1 relative to fundamental'},
    26: {'unit': '%', 'short_desc': 'IN_h11', 'long_desc': '11th harmonic of neutral current IN relative to fundamental'},
    
    27: {'unit': '%', 'short_desc': 'I1_h13', 'long_desc': '13th harmonic of current I1 relative to fundamental'},
    28: {'unit': '%', 'short_desc': 'IN_h13', 'long_desc': '13th harmonic of neutral current IN relative to fundamental'},
    
    29: {'unit': '%', 'short_desc': 'I1_h15', 'long_desc': '15th harmonic of current I1 relative to fundamental'},
    30: {'unit': '%', 'short_desc': 'IN_h15', 'long_desc': '15th harmonic of neutral current IN relative to fundamental'},
    
    31: {'unit': '%', 'short_desc': 'I1_h17', 'long_desc': '17th harmonic of current I1 relative to fundamental'},
    32: {'unit': '%', 'short_desc': 'IN_h17', 'long_desc': '17th harmonic of neutral current IN relative to fundamental'},
    
    33: {'unit': '%', 'short_desc': 'I1_h19', 'long_desc': '19th harmonic of current I1 relative to fundamental'},
    34: {'unit': '%', 'short_desc': 'IN_h19', 'long_desc': '19th harmonic of neutral current IN relative to fundamental'},
    
    35: {'unit': '%', 'short_desc': 'I1_h21', 'long_desc': '21th harmonic of current I1 relative to fundamental'},
    36: {'unit': '%', 'short_desc': 'IN_h21', 'long_desc': '21th harmonic of neutral current IN relative to fundamental'},
    
    37: {'unit': '%', 'short_desc': 'I1_h23', 'long_desc': '23th harmonic of current I1 relative to fundamental'},
    38: {'unit': '%', 'short_desc': 'IN_h23', 'long_desc': '23th harmonic of neutral current IN relative to fundamental'},
    
    39: {'unit': '%', 'short_desc': 'I1_h25', 'long_desc': '25th harmonic of current I1 relative to fundamental'},
    40: {'unit': '%', 'short_desc': 'IN_h25', 'long_desc': '25th harmonic of neutral current IN relative to fundamental'},
    
    41: {'unit': '%', 'short_desc': 'I1_h27', 'long_desc': '27th harmonic of current I1 relative to fundamental'},
    42: {'unit': '%', 'short_desc': 'IN_h27', 'long_desc': '27th harmonic of neutral current IN relative to fundamental'},
    
    43: {'unit': '%', 'short_desc': 'I1_h29', 'long_desc': '29th harmonic of current I1 relative to fundamental'},
    44: {'unit': '%', 'short_desc': 'IN_h29', 'long_desc': '29th harmonic of neutral current IN relative to fundamental'},
    
    45: {'unit': '%', 'short_desc': 'I1_h31', 'long_desc': '31th harmonic of current I1 relative to fundamental'},
    46: {'unit': '%', 'short_desc': 'IN_h31', 'long_desc': '31th harmonic of neutral current IN relative to fundamental'},
    
    #47: {'unit': '0.10%', 'short_desc': 'I1_h33', 'long_desc': '33th harmonic of current I1 relative to fundamental'},
    #48: {'unit': '0.10%', 'short_desc': 'IN_h33', 'long_desc': '33th harmonic of neutral current IN relative to fundamental'},
    #49: {'unit': '0.10%', 'short_desc': 'I1_h35', 'long_desc': '35th harmonic of current I1 relative to fundamental'},
    #50: {'unit': '0.10%', 'short_desc': 'IN_h35', 'long_desc': '35th harmonic of neutral current IN relative to fundamental'},
    #51: {'unit': '0.10%', 'short_desc': 'I1_h37', 'long_desc': '37th harmonic of current I1 relative to fundamental'},
    #52: {'unit': '0.10%', 'short_desc': 'IN_h37', 'long_desc': '37th harmonic of neutral current IN relative to fundamental'},
    #53: {'unit': '0.10%', 'short_desc': 'I1_h39', 'long_desc': '39th harmonic of current I1 relative to fundamental'},
    #54: {'unit': '0.10%', 'short_desc': 'IN_h39', 'long_desc': '39th harmonic of neutral current IN relative to fundamental'},
    #55: {'unit': '0.10%', 'short_desc': 'I1_h41', 'long_desc': '41th harmonic of current I1 relative to fundamental'},
    #56: {'unit': '0.10%', 'short_desc': 'IN_h41', 'long_desc': '41th harmonic of neutral current IN relative to fundamental'},
    #57: {'unit': '0.10%', 'short_desc': 'I1_h43', 'long_desc': '43th harmonic of current I1 relative to fundamental'},
    #58: {'unit': '0.10%', 'short_desc': 'IN_h43', 'long_desc': '43th harmonic of neutral current IN relative to fundamental'},
    #59: {'unit': '0.10%', 'short_desc': 'I1_h45', 'long_desc': '45th harmonic of current I1 relative to fundamental'},
    #60: {'unit': '0.10%', 'short_desc': 'IN_h45', 'long_desc': '45th harmonic of neutral current IN relative to fundamental'},
    #61: {'unit': '0.10%', 'short_desc': 'I1_h47', 'long_desc': '47th harmonic of current I1 relative to fundamental'},
    #62: {'unit': '0.10%', 'short_desc': 'IN_h47', 'long_desc': '47th harmonic of neutral current IN relative to fundamental'},
    #63: {'unit': '0.10%', 'short_desc': 'I1_h49', 'long_desc': '49th harmonic of current I1 relative to fundamental'},
    #64: {'unit': '0.10%', 'short_desc': 'IN_h49', 'long_desc': '49th harmonic of neutral current IN relative to fundamental'},
    #65: {'unit': '0.10%', 'short_desc': 'I1_h51', 'long_desc': '51th harmonic of current I1 relative to fundamental'},
    #66: {'unit': '0.10%', 'short_desc': 'IN_h51', 'long_desc': '51th harmonic of neutral current IN relative to fundamental'},
    #67: {'unit': '0.10%', 'short_desc': 'I1_h53', 'long_desc': '53th harmonic of current I1 relative to fundamental'},
    #68: {'unit': '0.10%', 'short_desc': 'IN_h53', 'long_desc': '53th harmonic of neutral current IN relative to fundamental'},
    #69: {'unit': '0.10%', 'short_desc': 'I1_h55', 'long_desc': '55th harmonic of current I1 relative to fundamental'},
    #70: {'unit': '0.10%', 'short_desc': 'IN_h55', 'long_desc': '55th harmonic of neutral current IN relative to fundamental'},
    #71: {'unit': '0.10%', 'short_desc': 'I1_h57', 'long_desc': '57th harmonic of current I1 relative to fundamental'},
    #72: {'unit': '0.10%', 'short_desc': 'IN_h57', 'long_desc': '57th harmonic of neutral current IN relative to fundamental'},
    #73: {'unit': '0.10%', 'short_desc': 'I1_h59', 'long_desc': '59th harmonic of current I1 relative to fundamental'},
    #74: {'unit': '0.10%', 'short_desc': 'IN_h59', 'long_desc': '59th harmonic of neutral current IN relative to fundamental'},
    #75: {'unit': '0.10%', 'short_desc': 'I1_h61', 'long_desc': '61th harmonic of current I1 relative to fundamental'},
    #76: {'unit': '0.10%', 'short_desc': 'IN_h61', 'long_desc': '61th harmonic of neutral current IN relative to fundamental'},
    #77: {'unit': '0.10%', 'short_desc': 'I1_h63', 'long_desc': '63th harmonic of current I1 relative to fundamental'},
    #78: {'unit': '0.10%', 'short_desc': 'IN_h63', 'long_desc': '63th harmonic of neutral current IN relative to fundamental'},
    
    80: {'unit': '%', 'short_desc': 'U1_h3', 'long_desc': '3th harmonic of phase voltage U1 relative to fundamental'},
    81: {'unit': '%', 'short_desc': 'U1_h5', 'long_desc': '5th harmonic of phase voltage U1 relative to fundamental'},
    82: {'unit': '%', 'short_desc': 'U1_h7', 'long_desc': '7th harmonic of phase voltage U1 relative to fundamental'},
    83: {'unit': '%', 'short_desc': 'U1_h9', 'long_desc': '9th harmonic of phase voltage U1 relative to fundamental'},
    84: {'unit': '%', 'short_desc': 'U1_h11', 'long_desc': '11th harmonic of phase voltage U1 relative to fundamental'},
    85: {'unit': '%', 'short_desc': 'U1_h13', 'long_desc': '13th harmonic of phase voltage U1 relative to fundamental'},
    86: {'unit': '%', 'short_desc': 'U1_h15', 'long_desc': '15th harmonic of phase voltage U1 relative to fundamental'},
    87: {'unit': '%', 'short_desc': 'U1_h17', 'long_desc': '17th harmonic of phase voltage U1 relative to fundamental'},
    88: {'unit': '%', 'short_desc': 'U1_h19', 'long_desc': '19th harmonic of phase voltage U1 relative to fundamental'},
    89: {'unit': '%', 'short_desc': 'U1_h21', 'long_desc': '21th harmonic of phase voltage U1 relative to fundamental'},
    90: {'unit': '%', 'short_desc': 'U1_h23', 'long_desc': '23th harmonic of phase voltage U1 relative to fundamental'},
    91: {'unit': '%', 'short_desc': 'U1_h25', 'long_desc': '25th harmonic of phase voltage U1 relative to fundamental'},
    92: {'unit': '%', 'short_desc': 'U1_h27', 'long_desc': '27th harmonic of phase voltage U1 relative to fundamental'},
    93: {'unit': '%', 'short_desc': 'U1_h29', 'long_desc': '29th harmonic of phase voltage U1 relative to fundamental'},
    94: {'unit': '%', 'short_desc': 'U1_h31', 'long_desc': '31th harmonic of phase voltage U1 relative to fundamental'},
    
    #95: {'unit': '0.10%', 'short_desc': 'V1_h33', 'long_desc': '33th harmonic of phase voltage V1 relative to fundamental'},
    #96: {'unit': '0.10%', 'short_desc': 'V1_h35', 'long_desc': '35th harmonic of phase voltage V1 relative to fundamental'},
    #97: {'unit': '0.10%', 'short_desc': 'V1_h37', 'long_desc': '37th harmonic of phase voltage V1 relative to fundamental'},
    #98: {'unit': '0.10%', 'short_desc': 'V1_h39', 'long_desc': '39th harmonic of phase voltage V1 relative to fundamental'},
    #99: {'unit': '0.10%', 'short_desc': 'V1_h41', 'long_desc': '41th harmonic of phase voltage V1 relative to fundamental'},
    #100: {'unit': '0.10%', 'short_desc': 'V1_h43', 'long_desc': '43th harmonic of phase voltage V1 relative to fundamental'},
    #101: {'unit': '0.10%', 'short_desc': 'V1_h45', 'long_desc': '45th harmonic of phase voltage V1 relative to fundamental'},
    #102: {'unit': '0.10%', 'short_desc': 'V1_h47', 'long_desc': '47th harmonic of phase voltage V1 relative to fundamental'},
    #103: {'unit': '0.10%', 'short_desc': 'V1_h49', 'long_desc': '49th harmonic of phase voltage V1 relative to fundamental'},
    #104: {'unit': '0.10%', 'short_desc': 'V1_h51', 'long_desc': '51th harmonic of phase voltage V1 relative to fundamental'},
    #105: {'unit': '0.10%', 'short_desc': 'V1_h53', 'long_desc': '53th harmonic of phase voltage V1 relative to fundamental'},
    #106: {'unit': '0.10%', 'short_desc': 'V1_h55', 'long_desc': '55th harmonic of phase voltage V1 relative to fundamental'},
    #107: {'unit': '0.10%', 'short_desc': 'V1_h57', 'long_desc': '57th harmonic of phase voltage V1 relative to fundamental'},
    #108: {'unit': '0.10%', 'short_desc': 'V1_h59', 'long_desc': '59th harmonic of phase voltage V1 relative to fundamental'},
    #109: {'unit': '0.10%', 'short_desc': 'V1_h61', 'long_desc': '61th harmonic of phase voltage V1 relative to fundamental'},
    #110: {'unit': '0.10%', 'short_desc': 'V1_h63', 'long_desc': '63th harmonic of phase voltage V1 relative to fundamental'},
}

EPI_CHANNELS_3_MEASUREMENT_DICT  = {
    #0: {'unit': 'h', 'short_desc': 'OpHours', 'long_desc': 'Operating hours counter'},
    
    1: {'unit': 'V', 'short_desc': 'U12', 'long_desc': 'Effective value of the Line voltage between phase 1 and 2 (U12)'},
    2: {'unit': 'V', 'short_desc': 'U23', 'long_desc': 'Effective value of the Line voltage between phase 2 and 3 (U23)'},
    3: {'unit': 'V', 'short_desc': 'U31', 'long_desc': 'Effective value of the Line voltage between phase 3 and 1 (U31)'},
    4: {'unit': 'V', 'short_desc': 'U1', 'long_desc': 'Effective value of the Phase voltage of phase 1 to neutral (U1)'},
    5: {'unit': 'V', 'short_desc': 'U2', 'long_desc': 'Effective value of the Phase voltage of phase 2 to neutral (U2)'},
    6: {'unit': 'V', 'short_desc': 'U3', 'long_desc': 'Effective value of the Phase voltage of phase 3 to neutral (U3)'},
    
    7: {'unit': 'Hz', 'short_desc': 'Freq', 'long_desc': 'Network frequency'},
    
    8: {'unit': 'A', 'short_desc': 'I1', 'long_desc': 'Effective value of the Current in phase 1 (I1)'},
    9: {'unit': 'A', 'short_desc': 'I2', 'long_desc': 'Effective value of the Current in phase 2 (I2)'},
    10: {'unit': 'A', 'short_desc': 'I3', 'long_desc': 'Effective value of the Current in phase 3 (I3)'},
    11: {'unit': 'A', 'short_desc': 'IN', 'long_desc': 'Effective value of the Current in neutral conductor (IN)'},
    
    12: {'unit': 'W', 'short_desc': 'P_total', 'long_desc': 'Total active power (DIN EN 61557-12 >0: Demand)'},
    13: {'unit': 'var', 'short_desc': 'Q_total_vec', 'long_desc': 'Total reactive power (vectorial sum, >0: Demand)'},
    14: {'unit': 'VA', 'short_desc': 'S_total_vec', 'long_desc': 'Total apparent power (vectorial sum)'},
    15: {'unit': '-', 'short_desc': 'PF_total_vec', 'long_desc': 'Power factor from vectorial power (>0: Inductive, <0: Capacitive)'},
    
    16: {'unit': 'W', 'short_desc': 'P1', 'long_desc': 'Active power in phase 1 (>0: Demand)'},
    17: {'unit': 'W', 'short_desc': 'P2', 'long_desc': 'Active power in phase 2 (>0: Demand)'},
    18: {'unit': 'W', 'short_desc': 'P3', 'long_desc': 'Active power in phase 3 (>0: Demand)'},
    19: {'unit': 'var', 'short_desc': 'Q1', 'long_desc': 'Reactive power in phase 1 (>0: Demand)'},
    20: {'unit': 'var', 'short_desc': 'Q2', 'long_desc': 'Reactive power in phase 2 (>0: Demand)'},
    21: {'unit': 'var', 'short_desc': 'Q3', 'long_desc': 'Reactive power in phase 3 (>0: Demand)'},
    22: {'unit': 'VA', 'short_desc': 'S1', 'long_desc': 'Apparent power in phase 1 (unsigned)'},
    23: {'unit': 'VA', 'short_desc': 'S2', 'long_desc': 'Apparent power in phase 2 (unsigned)'},
    24: {'unit': 'VA', 'short_desc': 'S3', 'long_desc': 'Apparent power in phase 3 (unsigned)'},
    
    25: {'unit': '-', 'short_desc': 'PF1', 'long_desc': 'Power factor of phase 1 (>0: Demand)'},
    26: {'unit': '-', 'short_desc': 'PF2', 'long_desc': 'Power factor of phase 2 (>0: Demand)'},
    27: {'unit': '-', 'short_desc': 'PF3', 'long_desc': 'Power factor of phase 3 (>0: Demand)'},
    
    #28: {'unit': 'kWh', 'short_desc': 'Ea_pos', 'long_desc': 'Current positive active energy'},
    #29: {'unit': 'kvarh', 'short_desc': 'Er_pos', 'long_desc': 'Current positive reactive energy'},
    #30: {'unit': 'kVAh', 'short_desc': 'Es', 'long_desc': 'Current apparent energy'},
    #31: {'unit': 'kWh', 'short_desc': 'Ea_neg', 'long_desc': 'Current negative active energy'},
    #32: {'unit': 'kvarh', 'short_desc': 'Er_neg', 'long_desc': 'Current negative reactive energy'},
    
    33: {'unit': 'A', 'short_desc': 'I_sys', 'long_desc': 'Average phase current (I1+I2+I3)/3'},
    34: {'unit': 'V', 'short_desc': 'U_line_avg', 'long_desc': 'Average line voltage (U12+U23+U31)/3'},
    35: {'unit': 'V', 'short_desc': 'U_phase_avg', 'long_desc': 'Average phase voltage (V1+V2+V3)/3'},
    
    36: {'unit': '%', 'short_desc': 'THD_U12', 'long_desc': 'Total Harmonic Distortion of line voltage U12'},
    37: {'unit': '%', 'short_desc': 'THD_U23', 'long_desc': 'Total Harmonic Distortion of line voltage U23'},
    38: {'unit': '%', 'short_desc': 'THD_U31', 'long_desc': 'Total Harmonic Distortion of line voltage U31'},
    39: {'unit': '%', 'short_desc': 'THD_U1', 'long_desc': 'Total Harmonic Distortion of phase voltage U1'},
    40: {'unit': '%', 'short_desc': 'THD_U2', 'long_desc': 'Total Harmonic Distortion of phase voltage U2'},
    41: {'unit': '%', 'short_desc': 'THD_U3', 'long_desc': 'Total Harmonic Distortion of phase voltage U3'},
    42: {'unit': '%', 'short_desc': 'THD_I1', 'long_desc': 'Total Harmonic Distortion of current I1'},
    43: {'unit': '%', 'short_desc': 'THD_I2', 'long_desc': 'Total Harmonic Distortion of current I2'},
    44: {'unit': '%', 'short_desc': 'THD_I3', 'long_desc': 'Total Harmonic Distortion of current I3'},
    45: {'unit': '%', 'short_desc': 'THD_IN', 'long_desc': 'Total Harmonic Distortion of neutral current IN'},
    
    #46: {'unit': '-', 'short_desc': 'Harm_Max', 'long_desc': 'Highest detectable harmonic (63)'},
    
    47: {'unit': '%', 'short_desc': 'I1_h3', 'long_desc': '3th harmonic of I1 relative to fundamental'},
    48: {'unit': '%', 'short_desc': 'I2_h3', 'long_desc': '3th harmonic of I2 relative to fundamental'},
    49: {'unit': '%', 'short_desc': 'I3_h3', 'long_desc': '3th harmonic of I3 relative to fundamental'},
    50: {'unit': '%', 'short_desc': 'IN_h3', 'long_desc': '3th harmonic of IN relative to fundamental'},
    
    51: {'unit': '%', 'short_desc': 'I1_h5', 'long_desc': '5th harmonic of I1 relative to fundamental'},
    52: {'unit': '%', 'short_desc': 'I2_h5', 'long_desc': '5th harmonic of I2 relative to fundamental'},
    53: {'unit': '%', 'short_desc': 'I3_h5', 'long_desc': '5th harmonic of I3 relative to fundamental'},
    54: {'unit': '%', 'short_desc': 'IN_h5', 'long_desc': '5th harmonic of IN relative to fundamental'},
    
    55: {'unit': '%', 'short_desc': 'I1_h7', 'long_desc': '7th harmonic of I1 relative to fundamental'},
    56: {'unit': '%', 'short_desc': 'I2_h7', 'long_desc': '7th harmonic of I2 relative to fundamental'},
    57: {'unit': '%', 'short_desc': 'I3_h7', 'long_desc': '7th harmonic of I3 relative to fundamental'},
    58: {'unit': '%', 'short_desc': 'IN_h7', 'long_desc': '7th harmonic of IN relative to fundamental'},
    
    59: {'unit': '%', 'short_desc': 'I1_h9', 'long_desc': '9th harmonic of I1 relative to fundamental'},
    60: {'unit': '%', 'short_desc': 'I2_h9', 'long_desc': '9th harmonic of I2 relative to fundamental'},
    61: {'unit': '%', 'short_desc': 'I3_h9', 'long_desc': '9th harmonic of I3 relative to fundamental'},
    62: {'unit': '%', 'short_desc': 'IN_h9', 'long_desc': '9th harmonic of IN relative to fundamental'},
    
    63: {'unit': '%', 'short_desc': 'I1_h11', 'long_desc': '11th harmonic of I1 relative to fundamental'},
    64: {'unit': '%', 'short_desc': 'I2_h11', 'long_desc': '11th harmonic of I2 relative to fundamental'},
    65: {'unit': '%', 'short_desc': 'I3_h11', 'long_desc': '11th harmonic of I3 relative to fundamental'},
    66: {'unit': '%', 'short_desc': 'IN_h11', 'long_desc': '11th harmonic of IN relative to fundamental'},
    
    67: {'unit': '%', 'short_desc': 'I1_h13', 'long_desc': '13th harmonic of I1 relative to fundamental'},
    68: {'unit': '%', 'short_desc': 'I2_h13', 'long_desc': '13th harmonic of I2 relative to fundamental'},
    69: {'unit': '%', 'short_desc': 'I3_h13', 'long_desc': '13th harmonic of I3 relative to fundamental'},
    70: {'unit': '%', 'short_desc': 'IN_h13', 'long_desc': '13th harmonic of IN relative to fundamental'},
    
    71: {'unit': '%', 'short_desc': 'I1_h15', 'long_desc': '15th harmonic of I1 relative to fundamental'},
    72: {'unit': '%', 'short_desc': 'I2_h15', 'long_desc': '15th harmonic of I2 relative to fundamental'},
    73: {'unit': '%', 'short_desc': 'I3_h15', 'long_desc': '15th harmonic of I3 relative to fundamental'},
    74: {'unit': '%', 'short_desc': 'IN_h15', 'long_desc': '15th harmonic of IN relative to fundamental'},
    
    75: {'unit': '%', 'short_desc': 'I1_h17', 'long_desc': '17th harmonic of I1 relative to fundamental'},
    76: {'unit': '%', 'short_desc': 'I2_h17', 'long_desc': '17th harmonic of I2 relative to fundamental'},
    77: {'unit': '%', 'short_desc': 'I3_h17', 'long_desc': '17th harmonic of I3 relative to fundamental'},
    78: {'unit': '%', 'short_desc': 'IN_h17', 'long_desc': '17th harmonic of IN relative to fundamental'},
    
    79: {'unit': '%', 'short_desc': 'I1_h19', 'long_desc': '19th harmonic of I1 relative to fundamental'},
    80: {'unit': '%', 'short_desc': 'I2_h19', 'long_desc': '19th harmonic of I2 relative to fundamental'},
    81: {'unit': '%', 'short_desc': 'I3_h19', 'long_desc': '19th harmonic of I3 relative to fundamental'},
    82: {'unit': '%', 'short_desc': 'IN_h19', 'long_desc': '19th harmonic of IN relative to fundamental'},
    
    83: {'unit': '%', 'short_desc': 'I1_h21', 'long_desc': '21th harmonic of I1 relative to fundamental'},
    84: {'unit': '%', 'short_desc': 'I2_h21', 'long_desc': '21th harmonic of I2 relative to fundamental'},
    85: {'unit': '%', 'short_desc': 'I3_h21', 'long_desc': '21th harmonic of I3 relative to fundamental'},
    86: {'unit': '%', 'short_desc': 'IN_h21', 'long_desc': '21th harmonic of IN relative to fundamental'},
    
    87: {'unit': '%', 'short_desc': 'I1_h23', 'long_desc': '23th harmonic of I1 relative to fundamental'},
    88: {'unit': '%', 'short_desc': 'I2_h23', 'long_desc': '23th harmonic of I2 relative to fundamental'},
    89: {'unit': '%', 'short_desc': 'I3_h23', 'long_desc': '23th harmonic of I3 relative to fundamental'},
    90: {'unit': '%', 'short_desc': 'IN_h23', 'long_desc': '23th harmonic of IN relative to fundamental'},
    
    91: {'unit': '%', 'short_desc': 'I1_h25', 'long_desc': '25th harmonic of I1 relative to fundamental'},
    92: {'unit': '%', 'short_desc': 'I2_h25', 'long_desc': '25th harmonic of I2 relative to fundamental'},
    93: {'unit': '%', 'short_desc': 'I3_h25', 'long_desc': '25th harmonic of I3 relative to fundamental'},
    94: {'unit': '%', 'short_desc': 'IN_h25', 'long_desc': '25th harmonic of IN relative to fundamental'},
    
    95: {'unit': '%', 'short_desc': 'I1_h27', 'long_desc': '27th harmonic of I1 relative to fundamental'},
    96: {'unit': '%', 'short_desc': 'I2_h27', 'long_desc': '27th harmonic of I2 relative to fundamental'},
    97: {'unit': '%', 'short_desc': 'I3_h27', 'long_desc': '27th harmonic of I3 relative to fundamental'},
    98: {'unit': '%', 'short_desc': 'IN_h27', 'long_desc': '27th harmonic of IN relative to fundamental'},
    
    99: {'unit': '%', 'short_desc': 'I1_h29', 'long_desc': '29th harmonic of I1 relative to fundamental'},
    100: {'unit': '%', 'short_desc': 'I2_h29', 'long_desc': '29th harmonic of I2 relative to fundamental'},
    101: {'unit': '%', 'short_desc': 'I3_h29', 'long_desc': '29th harmonic of I3 relative to fundamental'},
    102: {'unit': '%', 'short_desc': 'IN_h29', 'long_desc': '29th harmonic of IN relative to fundamental'},
    
    103: {'unit': '%', 'short_desc': 'I1_h31', 'long_desc': '31th harmonic of I1 relative to fundamental'},
    104: {'unit': '%', 'short_desc': 'I2_h31', 'long_desc': '31th harmonic of I2 relative to fundamental'},
    105: {'unit': '%', 'short_desc': 'I3_h31', 'long_desc': '31th harmonic of I3 relative to fundamental'},
    106: {'unit': '%', 'short_desc': 'IN_h31', 'long_desc': '31th harmonic of IN relative to fundamental'},
    
    #107: {'unit': '%', 'short_desc': 'I1_h33', 'long_desc': '33th harmonic of I1 relative to fundamental'},
    #108: {'unit': '%', 'short_desc': 'I2_h33', 'long_desc': '33th harmonic of I2 relative to fundamental'},
    #109: {'unit': '%', 'short_desc': 'I3_h33', 'long_desc': '33th harmonic of I3 relative to fundamental'},
    #110: {'unit': '%', 'short_desc': 'IN_h33', 'long_desc': '33th harmonic of IN relative to fundamental'},
    #111: {'unit': '%', 'short_desc': 'I1_h35', 'long_desc': '35th harmonic of I1 relative to fundamental'},
    #112: {'unit': '%', 'short_desc': 'I2_h35', 'long_desc': '35th harmonic of I2 relative to fundamental'},
    # 113: {'unit': '%', 'short_desc': 'I3_h35', 'long_desc': '35th harmonic of I3 relative to fundamental'},
    # 114: {'unit': '%', 'short_desc': 'IN_h35', 'long_desc': '35th harmonic of IN relative to fundamental'},
    # 115: {'unit': '%', 'short_desc': 'I1_h37', 'long_desc': '37th harmonic of I1 relative to fundamental'},
    # 116: {'unit': '%', 'short_desc': 'I2_h37', 'long_desc': '37th harmonic of I2 relative to fundamental'},
    # 117: {'unit': '%', 'short_desc': 'I3_h37', 'long_desc': '37th harmonic of I3 relative to fundamental'},
    # 118: {'unit': '%', 'short_desc': 'IN_h37', 'long_desc': '37th harmonic of IN relative to fundamental'},
    # 119: {'unit': '%', 'short_desc': 'I1_h39', 'long_desc': '39th harmonic of I1 relative to fundamental'},
    # 120: {'unit': '%', 'short_desc': 'I2_h39', 'long_desc': '39th harmonic of I2 relative to fundamental'},
    # 121: {'unit': '%', 'short_desc': 'I3_h39', 'long_desc': '39th harmonic of I3 relative to fundamental'},
    # 122: {'unit': '%', 'short_desc': 'IN_h39', 'long_desc': '39th harmonic of IN relative to fundamental'},
    # 123: {'unit': '%', 'short_desc': 'I1_h41', 'long_desc': '41th harmonic of I1 relative to fundamental'},
    # 124: {'unit': '%', 'short_desc': 'I2_h41', 'long_desc': '41th harmonic of I2 relative to fundamental'},
    # 125: {'unit': '%', 'short_desc': 'I3_h41', 'long_desc': '41th harmonic of I3 relative to fundamental'},
    # 126: {'unit': '%', 'short_desc': 'IN_h41', 'long_desc': '41th harmonic of IN relative to fundamental'},
    # 127: {'unit': '%', 'short_desc': 'I1_h43', 'long_desc': '43th harmonic of I1 relative to fundamental'},
    # 128: {'unit': '%', 'short_desc': 'I2_h43', 'long_desc': '43th harmonic of I2 relative to fundamental'},
    # 129: {'unit': '%', 'short_desc': 'I3_h43', 'long_desc': '43th harmonic of I3 relative to fundamental'},
    # 130: {'unit': '%', 'short_desc': 'IN_h43', 'long_desc': '43th harmonic of IN relative to fundamental'},
    # 131: {'unit': '%', 'short_desc': 'I1_h45', 'long_desc': '45th harmonic of I1 relative to fundamental'},
    # 132: {'unit': '%', 'short_desc': 'I2_h45', 'long_desc': '45th harmonic of I2 relative to fundamental'},
    # 133: {'unit': '%', 'short_desc': 'I3_h45', 'long_desc': '45th harmonic of I3 relative to fundamental'},
    # 134: {'unit': '%', 'short_desc': 'IN_h45', 'long_desc': '45th harmonic of IN relative to fundamental'},
    # 135: {'unit': '%', 'short_desc': 'I1_h47', 'long_desc': '47th harmonic of I1 relative to fundamental'},
    # 136: {'unit': '%', 'short_desc': 'I2_h47', 'long_desc': '47th harmonic of I2 relative to fundamental'},
    # 137: {'unit': '%', 'short_desc': 'I3_h47', 'long_desc': '47th harmonic of I3 relative to fundamental'},
    # 138: {'unit': '%', 'short_desc': 'IN_h47', 'long_desc': '47th harmonic of IN relative to fundamental'},
    # 139: {'unit': '%', 'short_desc': 'I1_h49', 'long_desc': '49th harmonic of I1 relative to fundamental'},
    # 140: {'unit': '%', 'short_desc': 'I2_h49', 'long_desc': '49th harmonic of I2 relative to fundamental'},
    # 141: {'unit': '%', 'short_desc': 'I3_h49', 'long_desc': '49th harmonic of I3 relative to fundamental'},
    # 142: {'unit': '%', 'short_desc': 'IN_h49', 'long_desc': '49th harmonic of IN relative to fundamental'},
    # 143: {'unit': '%', 'short_desc': 'I1_h51', 'long_desc': '51th harmonic of I1 relative to fundamental'},
    # 144: {'unit': '%', 'short_desc': 'I2_h51', 'long_desc': '51th harmonic of I2 relative to fundamental'},
    # 145: {'unit': '%', 'short_desc': 'I3_h51', 'long_desc': '51th harmonic of I3 relative to fundamental'},
    # 146: {'unit': '%', 'short_desc': 'IN_h51', 'long_desc': '51th harmonic of IN relative to fundamental'},
    # 147: {'unit': '%', 'short_desc': 'I1_h53', 'long_desc': '53th harmonic of I1 relative to fundamental'},
    # 148: {'unit': '%', 'short_desc': 'I2_h53', 'long_desc': '53th harmonic of I2 relative to fundamental'},
    # 149: {'unit': '%', 'short_desc': 'I3_h53', 'long_desc': '53th harmonic of I3 relative to fundamental'},
    # 150: {'unit': '%', 'short_desc': 'IN_h53', 'long_desc': '53th harmonic of IN relative to fundamental'},
    # 151: {'unit': '%', 'short_desc': 'I1_h55', 'long_desc': '55th harmonic of I1 relative to fundamental'},
    # 152: {'unit': '%', 'short_desc': 'I2_h55', 'long_desc': '55th harmonic of I2 relative to fundamental'},
    # 153: {'unit': '%', 'short_desc': 'I3_h55', 'long_desc': '55th harmonic of I3 relative to fundamental'},
    # 154: {'unit': '%', 'short_desc': 'IN_h55', 'long_desc': '55th harmonic of IN relative to fundamental'},
    # 155: {'unit': '%', 'short_desc': 'I1_h57', 'long_desc': '57th harmonic of I1 relative to fundamental'},
    # 156: {'unit': '%', 'short_desc': 'I2_h57', 'long_desc': '57th harmonic of I2 relative to fundamental'},
    # 157: {'unit': '%', 'short_desc': 'I3_h57', 'long_desc': '57th harmonic of I3 relative to fundamental'},
    # 158: {'unit': '%', 'short_desc': 'IN_h57', 'long_desc': '57th harmonic of IN relative to fundamental'},
    # 159: {'unit': '%', 'short_desc': 'I1_h59', 'long_desc': '59th harmonic of I1 relative to fundamental'},
    # 160: {'unit': '%', 'short_desc': 'I2_h59', 'long_desc': '59th harmonic of I2 relative to fundamental'},
    # 161: {'unit': '%', 'short_desc': 'I3_h59', 'long_desc': '59th harmonic of I3 relative to fundamental'},
    # 162: {'unit': '%', 'short_desc': 'IN_h59', 'long_desc': '59th harmonic of IN relative to fundamental'},
    # 163: {'unit': '%', 'short_desc': 'I1_h61', 'long_desc': '61th harmonic of I1 relative to fundamental'},
    # 164: {'unit': '%', 'short_desc': 'I2_h61', 'long_desc': '61th harmonic of I2 relative to fundamental'},
    # 165: {'unit': '%', 'short_desc': 'I3_h61', 'long_desc': '61th harmonic of I3 relative to fundamental'},
    # 166: {'unit': '%', 'short_desc': 'IN_h61', 'long_desc': '61th harmonic of IN relative to fundamental'},
    # 167: {'unit': '%', 'short_desc': 'I1_h63', 'long_desc': '63th harmonic of I1 relative to fundamental'},
    # 168: {'unit': '%', 'short_desc': 'I2_h63', 'long_desc': '63th harmonic of I2 relative to fundamental'},
    # 169: {'unit': '%', 'short_desc': 'I3_h63', 'long_desc': '63th harmonic of I3 relative to fundamental'},
    # 170: {'unit': '%', 'short_desc': 'IN_h63', 'long_desc': '63th harmonic of IN relative to fundamental'},
    # 171: ({'unit': '-', 'short_desc': 'Harm_Max', 'long_desc': 'Highest detectable harmonic (63)'},),
    
    172: {'unit': '%', 'short_desc': 'U12_h3', 'long_desc': '3th harmonic of U12 relative to fundamental'},
    173: {'unit': '%', 'short_desc': 'U23_h3', 'long_desc': '3th harmonic of U23 relative to fundamental'},
    174: {'unit': '%', 'short_desc': 'U31_h3', 'long_desc': '3th harmonic of U31 relative to fundamental'},
    
    175: {'unit': '%', 'short_desc': 'U12_h5', 'long_desc': '5th harmonic of U12 relative to fundamental'},
    176: {'unit': '%', 'short_desc': 'U23_h5', 'long_desc': '5th harmonic of U23 relative to fundamental'},
    177: {'unit': '%', 'short_desc': 'U31_h5', 'long_desc': '5th harmonic of U31 relative to fundamental'},
    
    178: {'unit': '%', 'short_desc': 'U12_h7', 'long_desc': '7th harmonic of U12 relative to fundamental'},
    179: {'unit': '%', 'short_desc': 'U23_h7', 'long_desc': '7th harmonic of U23 relative to fundamental'},
    180: {'unit': '%', 'short_desc': 'U31_h7', 'long_desc': '7th harmonic of U31 relative to fundamental'},
    
    181: {'unit': '%', 'short_desc': 'U12_h9', 'long_desc': '9th harmonic of U12 relative to fundamental'},
    182: {'unit': '%', 'short_desc': 'U23_h9', 'long_desc': '9th harmonic of U23 relative to fundamental'},
    183: {'unit': '%', 'short_desc': 'U31_h9', 'long_desc': '9th harmonic of U31 relative to fundamental'},
    
    184: {'unit': '%', 'short_desc': 'U12_h11', 'long_desc': '11th harmonic of U12 relative to fundamental'},
    185: {'unit': '%', 'short_desc': 'U23_h11', 'long_desc': '11th harmonic of U23 relative to fundamental'},
    186: {'unit': '%', 'short_desc': 'U31_h11', 'long_desc': '11th harmonic of U31 relative to fundamental'},
    
    187: {'unit': '%', 'short_desc': 'U12_h13', 'long_desc': '13th harmonic of U12 relative to fundamental'},
    188: {'unit': '%', 'short_desc': 'U23_h13', 'long_desc': '13th harmonic of U23 relative to fundamental'},
    189: {'unit': '%', 'short_desc': 'U31_h13', 'long_desc': '13th harmonic of U31 relative to fundamental'},
    
    190: {'unit': '%', 'short_desc': 'U12_h15', 'long_desc': '15th harmonic of U12 relative to fundamental'},
    191: {'unit': '%', 'short_desc': 'U23_h15', 'long_desc': '15th harmonic of U23 relative to fundamental'},
    192: {'unit': '%', 'short_desc': 'U31_h15', 'long_desc': '15th harmonic of U31 relative to fundamental'},
    
    193: {'unit': '%', 'short_desc': 'U12_h17', 'long_desc': '17th harmonic of U12 relative to fundamental'},
    194: {'unit': '%', 'short_desc': 'U23_h17', 'long_desc': '17th harmonic of U23 relative to fundamental'},
    195: {'unit': '%', 'short_desc': 'U31_h17', 'long_desc': '17th harmonic of U31 relative to fundamental'},
    
    196: {'unit': '%', 'short_desc': 'U12_h19', 'long_desc': '19th harmonic of U12 relative to fundamental'},
    197: {'unit': '%', 'short_desc': 'U23_h19', 'long_desc': '19th harmonic of U23 relative to fundamental'},
    198: {'unit': '%', 'short_desc': 'U31_h19', 'long_desc': '19th harmonic of U31 relative to fundamental'},
    
    199: {'unit': '%', 'short_desc': 'U12_h21', 'long_desc': '21th harmonic of U12 relative to fundamental'},
    200: {'unit': '%', 'short_desc': 'U23_h21', 'long_desc': '21th harmonic of U23 relative to fundamental'},
    201: {'unit': '%', 'short_desc': 'U31_h21', 'long_desc': '21th harmonic of U31 relative to fundamental'},
    
    202: {'unit': '%', 'short_desc': 'U12_h23', 'long_desc': '23th harmonic of U12 relative to fundamental'},
    203: {'unit': '%', 'short_desc': 'U23_h23', 'long_desc': '23th harmonic of U23 relative to fundamental'},
    204: {'unit': '%', 'short_desc': 'U31_h23', 'long_desc': '23th harmonic of U31 relative to fundamental'},
    
    205: {'unit': '%', 'short_desc': 'U12_h25', 'long_desc': '25th harmonic of U12 relative to fundamental'},
    206: {'unit': '%', 'short_desc': 'U23_h25', 'long_desc': '25th harmonic of U23 relative to fundamental'},
    207: {'unit': '%', 'short_desc': 'U31_h25', 'long_desc': '25th harmonic of U31 relative to fundamental'},
    
    208: {'unit': '%', 'short_desc': 'U12_h27', 'long_desc': '27th harmonic of U12 relative to fundamental'},
    209: {'unit': '%', 'short_desc': 'U23_h27', 'long_desc': '27th harmonic of U23 relative to fundamental'},
    210: {'unit': '%', 'short_desc': 'U31_h27', 'long_desc': '27th harmonic of U31 relative to fundamental'},
    
    211: {'unit': '%', 'short_desc': 'U12_h29', 'long_desc': '29th harmonic of U12 relative to fundamental'},
    212: {'unit': '%', 'short_desc': 'U23_h29', 'long_desc': '29th harmonic of U23 relative to fundamental'},
    213: {'unit': '%', 'short_desc': 'U31_h29', 'long_desc': '29th harmonic of U31 relative to fundamental'},
    
    214: {'unit': '%', 'short_desc': 'U12_h31', 'long_desc': '31th harmonic of U12 relative to fundamental'},
    215: {'unit': '%', 'short_desc': 'U23_h31', 'long_desc': '31th harmonic of U23 relative to fundamental'},
    216: {'unit': '%', 'short_desc': 'U31_h31', 'long_desc': '31th harmonic of U31 relative to fundamental'},
    
    # 217: {'unit': '%', 'short_desc': 'U12_h33', 'long_desc': '33th harmonic of U12 relative to fundamental'},
    # 218: {'unit': '%', 'short_desc': 'U23_h33', 'long_desc': '33th harmonic of U23 relative to fundamental'},
    # 219: {'unit': '%', 'short_desc': 'U31_h33', 'long_desc': '33th harmonic of U31 relative to fundamental'},
    # 220: {'unit': '%', 'short_desc': 'U12_h35', 'long_desc': '35th harmonic of U12 relative to fundamental'},
    # 221: {'unit': '%', 'short_desc': 'U23_h35', 'long_desc': '35th harmonic of U23 relative to fundamental'},
    # 222: {'unit': '%', 'short_desc': 'U31_h35', 'long_desc': '35th harmonic of U31 relative to fundamental'},
    # 223: {'unit': '%', 'short_desc': 'U12_h37', 'long_desc': '37th harmonic of U12 relative to fundamental'},
    # 224: {'unit': '%', 'short_desc': 'U23_h37', 'long_desc': '37th harmonic of U23 relative to fundamental'},
    # 225: {'unit': '%', 'short_desc': 'U31_h37', 'long_desc': '37th harmonic of U31 relative to fundamental'},
    # 226: {'unit': '%', 'short_desc': 'U12_h39', 'long_desc': '39th harmonic of U12 relative to fundamental'},
    # 227: {'unit': '%', 'short_desc': 'U23_h39', 'long_desc': '39th harmonic of U23 relative to fundamental'},
    # 228: {'unit': '%', 'short_desc': 'U31_h39', 'long_desc': '39th harmonic of U31 relative to fundamental'},
    # 229: {'unit': '%', 'short_desc': 'U12_h41', 'long_desc': '41th harmonic of U12 relative to fundamental'},
    # 230: {'unit': '%', 'short_desc': 'U23_h41', 'long_desc': '41th harmonic of U23 relative to fundamental'},
    # 231: {'unit': '%', 'short_desc': 'U31_h41', 'long_desc': '41th harmonic of U31 relative to fundamental'},
    # 232: {'unit': '%', 'short_desc': 'U12_h43', 'long_desc': '43th harmonic of U12 relative to fundamental'},
    # 233: {'unit': '%', 'short_desc': 'U23_h43', 'long_desc': '43th harmonic of U23 relative to fundamental'},
    # 234: {'unit': '%', 'short_desc': 'U31_h43', 'long_desc': '43th harmonic of U31 relative to fundamental'},
    # 235: {'unit': '%', 'short_desc': 'U12_h45', 'long_desc': '45th harmonic of U12 relative to fundamental'},
    # 236: {'unit': '%', 'short_desc': 'U23_h45', 'long_desc': '45th harmonic of U23 relative to fundamental'},
    # 237: {'unit': '%', 'short_desc': 'U31_h45', 'long_desc': '45th harmonic of U31 relative to fundamental'},
    # 238: {'unit': '%', 'short_desc': 'U12_h47', 'long_desc': '47th harmonic of U12 relative to fundamental'},
    # 239: {'unit': '%', 'short_desc': 'U23_h47', 'long_desc': '47th harmonic of U23 relative to fundamental'},
    # 240: {'unit': '%', 'short_desc': 'U31_h47', 'long_desc': '47th harmonic of U31 relative to fundamental'},
    # 241: {'unit': '%', 'short_desc': 'U12_h49', 'long_desc': '49th harmonic of U12 relative to fundamental'},
    # 242: {'unit': '%', 'short_desc': 'U23_h49', 'long_desc': '49th harmonic of U23 relative to fundamental'},
    # 243: {'unit': '%', 'short_desc': 'U31_h49', 'long_desc': '49th harmonic of U31 relative to fundamental'},
    # 244: {'unit': '%', 'short_desc': 'U12_h51', 'long_desc': '51th harmonic of U12 relative to fundamental'},
    # 245: {'unit': '%', 'short_desc': 'U23_h51', 'long_desc': '51th harmonic of U23 relative to fundamental'},
    # 246: {'unit': '%', 'short_desc': 'U31_h51', 'long_desc': '51th harmonic of U31 relative to fundamental'},
    # 247: {'unit': '%', 'short_desc': 'U12_h53', 'long_desc': '53th harmonic of U12 relative to fundamental'},
    # 248: {'unit': '%', 'short_desc': 'U23_h53', 'long_desc': '53th harmonic of U23 relative to fundamental'},
    # 249: {'unit': '%', 'short_desc': 'U31_h53', 'long_desc': '53th harmonic of U31 relative to fundamental'},
    # 250: {'unit': '%', 'short_desc': 'U12_h55', 'long_desc': '55th harmonic of U12 relative to fundamental'},
    # 251: {'unit': '%', 'short_desc': 'U23_h55', 'long_desc': '55th harmonic of U23 relative to fundamental'},
    # 252: {'unit': '%', 'short_desc': 'U31_h55', 'long_desc': '55th harmonic of U31 relative to fundamental'},
    # 253: {'unit': '%', 'short_desc': 'U12_h57', 'long_desc': '57th harmonic of U12 relative to fundamental'},
    # 254: {'unit': '%', 'short_desc': 'U23_h57', 'long_desc': '57th harmonic of U23 relative to fundamental'},
    # 255: {'unit': '%', 'short_desc': 'U31_h57', 'long_desc': '57th harmonic of U31 relative to fundamental'},
    # 256: {'unit': '%', 'short_desc': 'U12_h59', 'long_desc': '59th harmonic of U12 relative to fundamental'},
    # 257: {'unit': '%', 'short_desc': 'U23_h59', 'long_desc': '59th harmonic of U23 relative to fundamental'},
    # 258: {'unit': '%', 'short_desc': 'U31_h59', 'long_desc': '59th harmonic of U31 relative to fundamental'},
    # 259: {'unit': '%', 'short_desc': 'U12_h61', 'long_desc': '61th harmonic of U12 relative to fundamental'},
    # 260: {'unit': '%', 'short_desc': 'U23_h61', 'long_desc': '61th harmonic of U23 relative to fundamental'},
    # 261: {'unit': '%', 'short_desc': 'U31_h61', 'long_desc': '61th harmonic of U31 relative to fundamental'},
    # 262: {'unit': '%', 'short_desc': 'U12_h63', 'long_desc': '63th harmonic of U12 relative to fundamental'},
    # 263: {'unit': '%', 'short_desc': 'U23_h63', 'long_desc': '63th harmonic of U23 relative to fundamental'},
    # 264: {'unit': '%', 'short_desc': 'U31_h63', 'long_desc': '63th harmonic of U31 relative to fundamental'},
    # 265: ({'unit': '-', 'short_desc': 'Harm_Max', 'long_desc': 'Highest detectable harmonic (63)'},),
    
    266: {'unit': '%', 'short_desc': 'U1_h3', 'long_desc': '3th harmonic of U1 relative to fundamental'},
    267: {'unit': '%', 'short_desc': 'U2_h3', 'long_desc': '3th harmonic of U2 relative to fundamental'},
    268: {'unit': '%', 'short_desc': 'U3_h3', 'long_desc': '3th harmonic of U3 relative to fundamental'},
    
    269: {'unit': '%', 'short_desc': 'U1_h5', 'long_desc': '5th harmonic of U1 relative to fundamental'},
    270: {'unit': '%', 'short_desc': 'U2_h5', 'long_desc': '5th harmonic of U2 relative to fundamental'},
    271: {'unit': '%', 'short_desc': 'U3_h5', 'long_desc': '5th harmonic of U3 relative to fundamental'},
    
    272: {'unit': '%', 'short_desc': 'U1_h7', 'long_desc': '7th harmonic of U1 relative to fundamental'},
    273: {'unit': '%', 'short_desc': 'U2_h7', 'long_desc': '7th harmonic of U2 relative to fundamental'},
    274: {'unit': '%', 'short_desc': 'U3_h7', 'long_desc': '7th harmonic of U3 relative to fundamental'},
    
    275: {'unit': '%', 'short_desc': 'U1_h9', 'long_desc': '9th harmonic of U1 relative to fundamental'},
    276: {'unit': '%', 'short_desc': 'U2_h9', 'long_desc': '9th harmonic of U2 relative to fundamental'},
    277: {'unit': '%', 'short_desc': 'U3_h9', 'long_desc': '9th harmonic of U3 relative to fundamental'},
    
    278: {'unit': '%', 'short_desc': 'U1_h11', 'long_desc': '11th harmonic of U1 relative to fundamental'},
    279: {'unit': '%', 'short_desc': 'U2_h11', 'long_desc': '11th harmonic of U2 relative to fundamental'},
    280: {'unit': '%', 'short_desc': 'U3_h11', 'long_desc': '11th harmonic of U3 relative to fundamental'},
    
    281: {'unit': '%', 'short_desc': 'U1_h13', 'long_desc': '13th harmonic of U1 relative to fundamental'},
    282: {'unit': '%', 'short_desc': 'U2_h13', 'long_desc': '13th harmonic of U2 relative to fundamental'},
    283: {'unit': '%', 'short_desc': 'U3_h13', 'long_desc': '13th harmonic of U3 relative to fundamental'},
    
    284: {'unit': '%', 'short_desc': 'U1_h15', 'long_desc': '15th harmonic of U1 relative to fundamental'},
    285: {'unit': '%', 'short_desc': 'U2_h15', 'long_desc': '15th harmonic of U2 relative to fundamental'},
    286: {'unit': '%', 'short_desc': 'U3_h15', 'long_desc': '15th harmonic of U3 relative to fundamental'},
    
    287: {'unit': '%', 'short_desc': 'U1_h17', 'long_desc': '17th harmonic of U1 relative to fundamental'},
    288: {'unit': '%', 'short_desc': 'U2_h17', 'long_desc': '17th harmonic of U2 relative to fundamental'},
    289: {'unit': '%', 'short_desc': 'U3_h17', 'long_desc': '17th harmonic of U3 relative to fundamental'},
    
    290: {'unit': '%', 'short_desc': 'U1_h19', 'long_desc': '19th harmonic of U1 relative to fundamental'},
    291: {'unit': '%', 'short_desc': 'U2_h19', 'long_desc': '19th harmonic of U2 relative to fundamental'},
    292: {'unit': '%', 'short_desc': 'U3_h19', 'long_desc': '19th harmonic of U3 relative to fundamental'},
    
    293: {'unit': '%', 'short_desc': 'U1_h21', 'long_desc': '21th harmonic of U1 relative to fundamental'},
    294: {'unit': '%', 'short_desc': 'U2_h21', 'long_desc': '21th harmonic of U2 relative to fundamental'},
    295: {'unit': '%', 'short_desc': 'U3_h21', 'long_desc': '21th harmonic of U3 relative to fundamental'},
    
    296: {'unit': '%', 'short_desc': 'U1_h23', 'long_desc': '23th harmonic of U1 relative to fundamental'},
    297: {'unit': '%', 'short_desc': 'U2_h23', 'long_desc': '23th harmonic of U2 relative to fundamental'},
    298: {'unit': '%', 'short_desc': 'U3_h23', 'long_desc': '23th harmonic of U3 relative to fundamental'},
    
    299: {'unit': '%', 'short_desc': 'U1_h25', 'long_desc': '25th harmonic of U1 relative to fundamental'},
    300: {'unit': '%', 'short_desc': 'U2_h25', 'long_desc': '25th harmonic of U2 relative to fundamental'},
    301: {'unit': '%', 'short_desc': 'U3_h25', 'long_desc': '25th harmonic of U3 relative to fundamental'},
    
    302: {'unit': '%', 'short_desc': 'U1_h27', 'long_desc': '27th harmonic of U1 relative to fundamental'},
    303: {'unit': '%', 'short_desc': 'U2_h27', 'long_desc': '27th harmonic of U2 relative to fundamental'},
    304: {'unit': '%', 'short_desc': 'U3_h27', 'long_desc': '27th harmonic of U3 relative to fundamental'},
    
    305: {'unit': '%', 'short_desc': 'U1_h29', 'long_desc': '29th harmonic of U1 relative to fundamental'},
    306: {'unit': '%', 'short_desc': 'U2_h29', 'long_desc': '29th harmonic of U2 relative to fundamental'},
    307: {'unit': '%', 'short_desc': 'U3_h29', 'long_desc': '29th harmonic of U3 relative to fundamental'},
    
    308: {'unit': '%', 'short_desc': 'U1_h31', 'long_desc': '31th harmonic of U1 relative to fundamental'},
    309: {'unit': '%', 'short_desc': 'U2_h31', 'long_desc': '31th harmonic of U2 relative to fundamental'},
    310: {'unit': '%', 'short_desc': 'U3_h31', 'long_desc': '31th harmonic of U3 relative to fundamental'},
    
    # 311: {'unit': '%', 'short_desc': 'U1_h33', 'long_desc': '33th harmonic of U1 relative to fundamental'},
    # 312: {'unit': '%', 'short_desc': 'U2_h33', 'long_desc': '33th harmonic of U2 relative to fundamental'},
    # 313: {'unit': '%', 'short_desc': 'U3_h33', 'long_desc': '33th harmonic of U3 relative to fundamental'},
    # 314: {'unit': '%', 'short_desc': 'U1_h35', 'long_desc': '35th harmonic of U1 relative to fundamental'},
    # 315: {'unit': '%', 'short_desc': 'U2_h35', 'long_desc': '35th harmonic of U2 relative to fundamental'},
    # 316: {'unit': '%', 'short_desc': 'U3_h35', 'long_desc': '35th harmonic of U3 relative to fundamental'},
    # 317: {'unit': '%', 'short_desc': 'U1_h37', 'long_desc': '37th harmonic of U1 relative to fundamental'},
    # 318: {'unit': '%', 'short_desc': 'U2_h37', 'long_desc': '37th harmonic of U2 relative to fundamental'},
    # 319: {'unit': '%', 'short_desc': 'U3_h37', 'long_desc': '37th harmonic of U3 relative to fundamental'},
    # 320: {'unit': '%', 'short_desc': 'U1_h39', 'long_desc': '39th harmonic of U1 relative to fundamental'},
    # 321: {'unit': '%', 'short_desc': 'U2_h39', 'long_desc': '39th harmonic of U2 relative to fundamental'},
    # 322: {'unit': '%', 'short_desc': 'U3_h39', 'long_desc': '39th harmonic of U3 relative to fundamental'},
    # 323: {'unit': '%', 'short_desc': 'U1_h41', 'long_desc': '41th harmonic of U1 relative to fundamental'},
    # 324: {'unit': '%', 'short_desc': 'U2_h41', 'long_desc': '41th harmonic of U2 relative to fundamental'},
    # 325: {'unit': '%', 'short_desc': 'U3_h41', 'long_desc': '41th harmonic of U3 relative to fundamental'},
    # 326: {'unit': '%', 'short_desc': 'U1_h43', 'long_desc': '43th harmonic of U1 relative to fundamental'},
    # 327: {'unit': '%', 'short_desc': 'U2_h43', 'long_desc': '43th harmonic of U2 relative to fundamental'},
    # 328: {'unit': '%', 'short_desc': 'U3_h43', 'long_desc': '43th harmonic of U3 relative to fundamental'},
    # 329: {'unit': '%', 'short_desc': 'U1_h45', 'long_desc': '45th harmonic of U1 relative to fundamental'},
    # 330: {'unit': '%', 'short_desc': 'U2_h45', 'long_desc': '45th harmonic of U2 relative to fundamental'},
    # 331: {'unit': '%', 'short_desc': 'U3_h45', 'long_desc': '45th harmonic of U3 relative to fundamental'},
    # 332: {'unit': '%', 'short_desc': 'U1_h47', 'long_desc': '47th harmonic of U1 relative to fundamental'},
    # 333: {'unit': '%', 'short_desc': 'U2_h47', 'long_desc': '47th harmonic of U2 relative to fundamental'},
    # 334: {'unit': '%', 'short_desc': 'U3_h47', 'long_desc': '47th harmonic of U3 relative to fundamental'},
    # 335: {'unit': '%', 'short_desc': 'U1_h49', 'long_desc': '49th harmonic of U1 relative to fundamental'},
    # 336: {'unit': '%', 'short_desc': 'U2_h49', 'long_desc': '49th harmonic of U2 relative to fundamental'},
    # 337: {'unit': '%', 'short_desc': 'U3_h49', 'long_desc': '49th harmonic of U3 relative to fundamental'},
    # 338: {'unit': '%', 'short_desc': 'U1_h51', 'long_desc': '51th harmonic of U1 relative to fundamental'},
    # 339: {'unit': '%', 'short_desc': 'U2_h51', 'long_desc': '51th harmonic of U2 relative to fundamental'},
    # 340: {'unit': '%', 'short_desc': 'U3_h51', 'long_desc': '51th harmonic of U3 relative to fundamental'},
    # 341: {'unit': '%', 'short_desc': 'U1_h53', 'long_desc': '53th harmonic of U1 relative to fundamental'},
    # 342: {'unit': '%', 'short_desc': 'U2_h53', 'long_desc': '53th harmonic of U2 relative to fundamental'},
    # 343: {'unit': '%', 'short_desc': 'U3_h53', 'long_desc': '53th harmonic of U3 relative to fundamental'},
    # 344: {'unit': '%', 'short_desc': 'U1_h55', 'long_desc': '55th harmonic of U1 relative to fundamental'},
    # 345: {'unit': '%', 'short_desc': 'U2_h55', 'long_desc': '55th harmonic of U2 relative to fundamental'},
    # 346: {'unit': '%', 'short_desc': 'U3_h55', 'long_desc': '55th harmonic of U3 relative to fundamental'},
    # 347: {'unit': '%', 'short_desc': 'U1_h57', 'long_desc': '57th harmonic of U1 relative to fundamental'},
    # 348: {'unit': '%', 'short_desc': 'U2_h57', 'long_desc': '57th harmonic of U2 relative to fundamental'},
    # 349: {'unit': '%', 'short_desc': 'U3_h57', 'long_desc': '57th harmonic of U3 relative to fundamental'},
    # 350: {'unit': '%', 'short_desc': 'U1_h59', 'long_desc': '59th harmonic of U1 relative to fundamental'},
    # 351: {'unit': '%', 'short_desc': 'U2_h59', 'long_desc': '59th harmonic of U2 relative to fundamental'},
    # 352: {'unit': '%', 'short_desc': 'U3_h59', 'long_desc': '59th harmonic of U3 relative to fundamental'},
    # 353: {'unit': '%', 'short_desc': 'U1_h61', 'long_desc': '61th harmonic of U1 relative to fundamental'},
    # 354: {'unit': '%', 'short_desc': 'U2_h61', 'long_desc': '61th harmonic of U2 relative to fundamental'},
    # 355: {'unit': '%', 'short_desc': 'U3_h61', 'long_desc': '61th harmonic of U3 relative to fundamental'},
    # 356: {'unit': '%', 'short_desc': 'U1_h63', 'long_desc': '63th harmonic of U1 relative to fundamental'},
    # 357: {'unit': '%', 'short_desc': 'U2_h63', 'long_desc': '63th harmonic of U2 relative to fundamental'},
    # 358: {'unit': '%', 'short_desc': 'U3_h63', 'long_desc': '63th harmonic of U3 relative to fundamental'},
}

TH_CHANNELS_MEASUREMENT_DICT = {
   1: {'unit': 'V', 'short_desc': 'U12', 'long_desc': 'Effective value of the Line voltage between phase 1 and 2 (U12)'},
    2: {'unit': 'V', 'short_desc': 'U23', 'long_desc': 'Effective value of the Line voltage between phase 2 and 3 (U23)'},
    3: {'unit': 'V', 'short_desc': 'U31', 'long_desc': 'Effective value of the Line voltage between phase 3 and 1 (U31)'},
    4: {'unit': 'V', 'short_desc': 'U1', 'long_desc': 'Effective value of the Phase voltage of phase 1 to neutral (U1)'},
    5: {'unit': 'V', 'short_desc': 'U2', 'long_desc': 'Effective value of the Phase voltage of phase 2 to neutral (U2)'},
    6: {'unit': 'V', 'short_desc': 'U3', 'long_desc': 'Effective value of the Phase voltage of phase 3 to neutral (U3)'},

    8: {'unit': 'Hz', 'short_desc': 'Freq', 'long_desc': 'Network frequency'},

    9: {'unit': 'A', 'short_desc': 'I1', 'long_desc': 'Effective value of the Current in phase 1 (I1)'},
    10: {'unit': 'A', 'short_desc': 'I2', 'long_desc': 'Effective value of the Current in phase 2 (I2)'},
    11: {'unit': 'A', 'short_desc': 'I3', 'long_desc': 'Effective value of the Current in phase 3 (I3)'},
    12: {'unit': 'A', 'short_desc': 'IN', 'long_desc': 'Effective value of the Current in neutral conductor (IN)'},

    13: {'unit': 'W', 'short_desc': 'P_total', 'long_desc': 'Total active power (DIN EN 61557-12 >0: Demand)'},
    14: {'unit': 'var', 'short_desc': 'Q_total', 'long_desc': 'Total reactive power (vectorial sum, >0: Demand)'},
    15: {'unit': 'VA', 'short_desc': 'S_total', 'long_desc': 'Total apparent power (vectorial sum)'},
    16: {'unit': '-', 'short_desc': 'PF_total', 'long_desc': 'Power factor from vectorial power (>0: Inductive, <0: Capacitive)'},

    17: {'unit': 'W', 'short_desc': 'P1', 'long_desc': 'Active power in phase 1 (>0: Demand)'},
    18: {'unit': 'W', 'short_desc': 'P2', 'long_desc': 'Active power in phase 2 (>0: Demand)'},
    19: {'unit': 'W', 'short_desc': 'P3', 'long_desc': 'Active power in phase 3 (>0: Demand)'},
    20: {'unit': 'var', 'short_desc': 'Q1', 'long_desc': 'Reactive power in phase 1 (>0: Demand)'},
    21: {'unit': 'var', 'short_desc': 'Q2', 'long_desc': 'Reactive power in phase 2 (>0: Demand)'},
    22: {'unit': 'var', 'short_desc': 'Q3', 'long_desc': 'Reactive power in phase 3 (>0: Demand)'},
    23: {'unit': 'VA', 'short_desc': 'S1', 'long_desc': 'Apparent power in phase 1 (unsigned)'},
    24: {'unit': 'VA', 'short_desc': 'S2', 'long_desc': 'Apparent power in phase 2 (unsigned)'},
    25: {'unit': 'VA', 'short_desc': 'S3', 'long_desc': 'Apparent power in phase 3 (unsigned)'},

    26: {'unit': '-', 'short_desc': 'PF1', 'long_desc': 'Power factor of phase 1 (>0: Demand)'},
    27: {'unit': '-', 'short_desc': 'PF2', 'long_desc': 'Power factor of phase 2 (>0: Demand)'},
    28: {'unit': '-', 'short_desc': 'PF3', 'long_desc': 'Power factor of phase 3 (>0: Demand)'},

    29: {'unit': '-', 'short_desc': 'LoadType1', 'long_desc': 'Load type phase 1: -1 capacitive, +1 inductive, 0 resistive'},
    30: {'unit': '-', 'short_desc': 'LoadType2', 'long_desc': 'Load type phase 2: -1 capacitive, +1 inductive, 0 resistive'},
    31: {'unit': '-', 'short_desc': 'LoadType3', 'long_desc': 'Load type phase 3: -1 capacitive, +1 inductive, 0 resistive'},

    32: {'unit': 'A', 'short_desc': 'I_sys', 'long_desc': 'Average phase current'},
    33: {'unit': 'V', 'short_desc': 'U_line_avg', 'long_desc': 'Average line voltage'},
    34: {'unit': 'V', 'short_desc': 'U_phase_avg', 'long_desc': 'Average phase voltage'},
    # 35: {'unit': 'var', 'short_desc': 'Q_total_arith', 'long_desc': 'Total reactive power (arithmetic sum)'},
    # 36: {'unit': 'VA', 'short_desc': 'S_total_arith', 'long_desc': 'Total apparent power (arithmetic sum)'},
    # 37: {'unit': '-', 'short_desc': 'PF_total_arith', 'long_desc': 'Total power factor (arithmetic sum)'},

    # 38: {'unit': '°', 'short_desc': 'Angle_UI1', 'long_desc': 'Phase angle between U and I in phase 1'},
    # 39: {'unit': '°', 'short_desc': 'Angle_UI2', 'long_desc': 'Phase angle between U and I in phase 2'},
    # 40: {'unit': '°', 'short_desc': 'Angle_UI3', 'long_desc': 'Phase angle between U and I in phase 3'},

    # 41: {'unit': '-', 'short_desc': 'cos_phi1', 'long_desc': 'cos(φ) of phase 1 (fundamental)'},
    # 42: {'unit': '-', 'short_desc': 'cos_phi2', 'long_desc': 'cos(φ) of phase 2 (fundamental)'},
    # 43: {'unit': '-', 'short_desc': 'cos_phi3', 'long_desc': 'cos(φ) of phase 3 (fundamental)'},

    44: {'unit': '°', 'short_desc': 'Angle_U1', 'long_desc': 'Voltage angle of phase U1'},
    45: {'unit': '°', 'short_desc': 'Angle_U2', 'long_desc': 'Voltage angle of phase U2'},
    46: {'unit': '°', 'short_desc': 'Angle_U3', 'long_desc': 'Voltage angle of phase U3'},

    47: {'unit': '-', 'short_desc': 'RotField', 'long_desc': 'Rotating field direction: 1 clockwise, -1 counterclockwise, 0 none'},
    48: {'unit': 'h', 'short_desc': 'OpHours', 'long_desc': 'Operating hours counted'},

    49: {'unit': '%', 'short_desc': 'THD_U12', 'long_desc': 'Total Harmonic Distortion of line voltage U12'},
    50: {'unit': '%', 'short_desc': 'THD_U23', 'long_desc': 'Total Harmonic Distortion of line voltage U23'},
    51: {'unit': '%', 'short_desc': 'THD_U31', 'long_desc': 'Total Harmonic Distortion of line voltage U31'},
    52: {'unit': '%', 'short_desc': 'THD_U1', 'long_desc': 'Total Harmonic Distortion of phase voltage U1'},
    53: {'unit': '%', 'short_desc': 'THD_U2', 'long_desc': 'Total Harmonic Distortion of phase voltage U2'},
    54: {'unit': '%', 'short_desc': 'THD_U3', 'long_desc': 'Total Harmonic Distortion of phase voltage U3'},

    55: {'unit': '%', 'short_desc': 'THD_I1', 'long_desc': 'Total Harmonic Distortion of current I1'},
    56: {'unit': '%', 'short_desc': 'THD_I2', 'long_desc': 'Total Harmonic Distortion of current I2'},
    57: {'unit': '%', 'short_desc': 'THD_I3', 'long_desc': 'Total Harmonic Distortion of current I3'},
    58: {'unit': '%', 'short_desc': 'THD_IN', 'long_desc': 'Total Harmonic Distortion of neutral current IN'},

    # 59: {'unit': 'V', 'short_desc': 'U1_RMS_fund', 'long_desc': 'RMS of fundamental component of phase voltage U1'},
    # 60: {'unit': '%', 'short_desc': 'U1_DC', 'long_desc': 'DC component of phase voltage U1 relative to fundamental'},
    # 61: {'unit': '%', 'short_desc': 'U1_fund', 'long_desc': 'Fundamental component of phase voltage U1 (100%)'},
    62: {'unit': '%', 'short_desc': 'U1_h2', 'long_desc': '2nd harmonic of phase voltage U1 relative to fundamental'},
    63: {'unit': '%', 'short_desc': 'U1_h3', 'long_desc': '3rd harmonic of phase voltage U1 relative to fundamental'},
    64: {'unit': '%', 'short_desc': 'U1_h4', 'long_desc': '4th harmonic of phase voltage U1 relative to fundamental'},
    65: {'unit': '%', 'short_desc': 'U1_h5', 'long_desc': '5th harmonic of phase voltage U1 relative to fundamental'},

    # 66: {'unit': 'V', 'short_desc': 'U2_RMS_fund', 'long_desc': 'RMS of fundamental component of phase voltage U2'},
    # 67: {'unit': '%', 'short_desc': 'U2_DC', 'long_desc': 'DC component of phase voltage U2 relative to fundamental'},
    # 68: {'unit': '%', 'short_desc': 'U2_fund', 'long_desc': 'Fundamental component of phase voltage U2 (100%)'},
    69: {'unit': '%', 'short_desc': 'U2_h2', 'long_desc': '2nd harmonic of phase voltage U2 relative to fundamental'},
    70: {'unit': '%', 'short_desc': 'U2_h3', 'long_desc': '3rd harmonic of phase voltage U2 relative to fundamental'},
    71: {'unit': '%', 'short_desc': 'U2_h4', 'long_desc': '4th harmonic of phase voltage U2 relative to fundamental'},
    72: {'unit': '%', 'short_desc': 'U2_h5', 'long_desc': '5th harmonic of phase voltage U2 relative to fundamental'},

    # 73: {'unit': 'V', 'short_desc': 'U3_RMS_fund', 'long_desc': 'RMS of fundamental component of phase voltage U3'},
    # 74: {'unit': '%', 'short_desc': 'U3_DC', 'long_desc': 'DC component of phase voltage U3 relative to fundamental'},
    # 75: {'unit': '%', 'short_desc': 'U3_fund', 'long_desc': 'Fundamental component of phase voltage U3 (100%)'},
    76: {'unit': '%', 'short_desc': 'U3_h2', 'long_desc': '2nd harmonic of phase voltage U3 relative to fundamental'},
    77: {'unit': '%', 'short_desc': 'U3_h3', 'long_desc': '3rd harmonic of phase voltage U3 relative to fundamental'},
    78: {'unit': '%', 'short_desc': 'U3_h4', 'long_desc': '4th harmonic of phase voltage U3 relative to fundamental'},
    79: {'unit': '%', 'short_desc': 'U3_h5', 'long_desc': '5th harmonic of phase voltage U3 relative to fundamental'},

    # 80: {'unit': 'V', 'short_desc': 'U12_RMS_fund', 'long_desc': 'RMS of fundamental component of line voltage U12'},
    # 81: {'unit': '%', 'short_desc': 'U12_DC', 'long_desc': 'DC component of line voltage U12 relative to fundamental'},
    # 82: {'unit': '%', 'short_desc': 'U12_fund', 'long_desc': 'Fundamental component of line voltage U12 (100%)'},
    83: {'unit': '%', 'short_desc': 'U12_h2', 'long_desc': '2nd harmonic of line voltage U12 relative to fundamental'},
    84: {'unit': '%', 'short_desc': 'U12_h3', 'long_desc': '3rd harmonic of line voltage U12 relative to fundamental'},
    85: {'unit': '%', 'short_desc': 'U12_h4', 'long_desc': '4th harmonic of line voltage U12 relative to fundamental'},
    86: {'unit': '%', 'short_desc': 'U12_h5', 'long_desc': '5th harmonic of line voltage U12 relative to fundamental'},

    # 87: {'unit': 'V', 'short_desc': 'U23_RMS_fund', 'long_desc': 'RMS of fundamental component of line voltage U23'},
    # 88: {'unit': '%', 'short_desc': 'U23_DC', 'long_desc': 'DC component of line voltage U23 relative to fundamental'},
    # 89: {'unit': '%', 'short_desc': 'U23_fund', 'long_desc': 'Fundamental component of line voltage U23 (100%)'},
    90: {'unit': '%', 'short_desc': 'U23_h2', 'long_desc': '2nd harmonic of line voltage U23 relative to fundamental'},
    91: {'unit': '%', 'short_desc': 'U23_h3', 'long_desc': '3rd harmonic of line voltage U23 relative to fundamental'},
    92: {'unit': '%', 'short_desc': 'U23_h4', 'long_desc': '4th harmonic of line voltage U23 relative to fundamental'},
    93: {'unit': '%', 'short_desc': 'U23_h5', 'long_desc': '5th harmonic of line voltage U23 relative to fundamental'},

    # 94: {'unit': 'V', 'short_desc': 'U31_RMS_fund', 'long_desc': 'RMS of fundamental component of line voltage U31'},
    # 95: {'unit': '%', 'short_desc': 'U31_DC', 'long_desc': 'DC component of line voltage U31 relative to fundamental'},
    # 96: {'unit': '%', 'short_desc': 'U31_fund', 'long_desc': 'Fundamental component of line voltage U31 (100%)'},
    97: {'unit': '%', 'short_desc': 'U31_h2', 'long_desc': '2nd harmonic of line voltage U31 relative to fundamental'},
    98: {'unit': '%', 'short_desc': 'U31_h3', 'long_desc': '3rd harmonic of line voltage U31 relative to fundamental'},
    99: {'unit': '%', 'short_desc': 'U31_h4', 'long_desc': '4th harmonic of line voltage U31 relative to fundamental'},
    100: {'unit': '%', 'short_desc': 'U31_h5', 'long_desc': '5th harmonic of line voltage U31 relative to fundamental'},

    # 101: {'unit': 'A', 'short_desc': 'I1_RMS_fund', 'long_desc': 'RMS of fundamental component of current I1'},
    # 102: {'unit': '%', 'short_desc': 'I1_DC', 'long_desc': 'DC component of current I1 relative to fundamental'},
    # 103: {'unit': '%', 'short_desc': 'I1_fund', 'long_desc': 'Fundamental component of current I1 (100%)'},
    104: {'unit': '%', 'short_desc': 'I1_h2', 'long_desc': '2nd harmonic of current I1 relative to fundamental'},
    105: {'unit': '%', 'short_desc': 'I1_h3', 'long_desc': '3rd harmonic of current I1 relative to fundamental'},
    106: {'unit': '%', 'short_desc': 'I1_h4', 'long_desc': '4th harmonic of current I1 relative to fundamental'},
    107: {'unit': '%', 'short_desc': 'I1_h5', 'long_desc': '5th harmonic of current I1 relative to fundamental'},

    # 108: {'unit': 'A', 'short_desc': 'I2_RMS_fund', 'long_desc': 'RMS of fundamental component of current I2'},
    # 109: {'unit': '%', 'short_desc': 'I2_DC', 'long_desc': 'DC component of current I2 relative to fundamental'},
    # 110: {'unit': '%', 'short_desc': 'I2_fund', 'long_desc': 'Fundamental component of current I2 (100%)'},
    111: {'unit': '%', 'short_desc': 'I2_h2', 'long_desc': '2nd harmonic of current I2 relative to fundamental'},
    112: {'unit': '%', 'short_desc': 'I2_h3', 'long_desc': '3rd harmonic of current I2 relative to fundamental'},
    113: {'unit': '%', 'short_desc': 'I2_h4', 'long_desc': '4th harmonic of current I2 relative to fundamental'},
    114: {'unit': '%', 'short_desc': 'I2_h5', 'long_desc': '5th harmonic of current I2 relative to fundamental'},

    # 115: {'unit': 'A', 'short_desc': 'I3_RMS_fund', 'long_desc': 'RMS of fundamental component of current I3'},
    # 116: {'unit': '%', 'short_desc': 'I3_DC', 'long_desc': 'DC component of current I3 relative to fundamental'},
    # 117: {'unit': '%', 'short_desc': 'I3_fund', 'long_desc': 'Fundamental component of current I3 (100%)'},
    118: {'unit': '%', 'short_desc': 'I3_h2', 'long_desc': '2nd harmonic of current I3 relative to fundamental'},
    119: {'unit': '%', 'short_desc': 'I3_h3', 'long_desc': '3rd harmonic of current I3 relative to fundamental'},
    120: {'unit': '%', 'short_desc': 'I3_h4', 'long_desc': '4th harmonic of current I3 relative to fundamental'},
    121: {'unit': '%', 'short_desc': 'I3_h5', 'long_desc': '5th harmonic of current I3 relative to fundamental'},

    # 122: {'unit': 'A', 'short_desc': 'IN_RMS_fund', 'long_desc': 'RMS of fundamental component of neutral current IN'},
    # 123: {'unit': '%', 'short_desc': 'IN_DC', 'long_desc': 'DC component of neutral current IN relative to fundamental'},
    # 124: {'unit': '%', 'short_desc': 'IN_fund', 'long_desc': 'Fundamental component of neutral current IN (100%)'},
    125: {'unit': '%', 'short_desc': 'IN_h2', 'long_desc': '2nd harmonic of neutral current IN relative to fundamental'},
    126: {'unit': '%', 'short_desc': 'IN_h3', 'long_desc': '3rd harmonic of neutral current IN relative to fundamental'},
    127: {'unit': '%', 'short_desc': 'IN_h4', 'long_desc': '4th harmonic of neutral current IN relative to fundamental'},
    128: {'unit': '%', 'short_desc': 'IN_h5', 'long_desc': '5th harmonic of neutral current IN relative to fundamental'},


    129: {'unit': 'V', 'short_desc': 'U12_f', 'long_desc': 'Fundamental component of line voltage U12'},
    130: {'unit': 'V', 'short_desc': 'U23_f', 'long_desc': 'Fundamental component of line voltage U23'},
    131: {'unit': 'V', 'short_desc': 'U31_f', 'long_desc': 'Fundamental component of line voltage U31'},

    132: {'unit': 'V', 'short_desc': 'U1_f', 'long_desc': 'Fundamental component of phase voltage U1'},
    133: {'unit': 'V', 'short_desc': 'U2_f', 'long_desc': 'Fundamental component of phase voltage U2'},
    134: {'unit': 'V', 'short_desc': 'U3_f', 'long_desc': 'Fundamental component of phase voltage U3'},

    135: {'unit': 'Hz', 'short_desc': 'Freq_f', 'long_desc': 'Frequency of the fundamental wave'},

    136: {'unit': 'A', 'short_desc': 'I1_f', 'long_desc': 'Fundamental component of current I1'},
    137: {'unit': 'A', 'short_desc': 'I2_f', 'long_desc': 'Fundamental component of current I2'},
    138: {'unit': 'A', 'short_desc': 'I3_f', 'long_desc': 'Fundamental component of current I3'},
    139: {'unit': 'A', 'short_desc': 'IN_f', 'long_desc': 'Fundamental component of neutral current IN'},

    140: {'unit': 'W', 'short_desc': 'P_total_f', 'long_desc': 'Total active power of the fundamental wave'},
    141: {'unit': 'var', 'short_desc': 'Q_total_f', 'long_desc': 'Total reactive power (vectorial) of the fundamental wave'},
    142: {'unit': 'VA', 'short_desc': 'S_total_f', 'long_desc': 'Total apparent power (vectorial) of the fundamental wave'},

    143: {'unit': '-', 'short_desc': 'PF_total_f', 'long_desc': 'Power factor of the fundamental wave'},

    144: {'unit': 'W', 'short_desc': 'P1_f', 'long_desc': 'Active power in phase 1 (fundamental wave)'},
    145: {'unit': 'W', 'short_desc': 'P2_f', 'long_desc': 'Active power in phase 2 (fundamental wave)'},
    146: {'unit': 'W', 'short_desc': 'P3_f', 'long_desc': 'Active power in phase 3 (fundamental wave)'},
    147: {'unit': 'var', 'short_desc': 'Q1_f', 'long_desc': 'Reactive power in phase 1 (fundamental wave)'},
    148: {'unit': 'var', 'short_desc': 'Q2_f', 'long_desc': 'Reactive power in phase 2 (fundamental wave)'},
    149: {'unit': 'var', 'short_desc': 'Q3_f', 'long_desc': 'Reactive power in phase 3 (fundamental wave)'},
    150: {'unit': 'VA', 'short_desc': 'S1_f', 'long_desc': 'Apparent power in phase 1 (fundamental wave)'},
    151: {'unit': 'VA', 'short_desc': 'S2_f', 'long_desc': 'Apparent power in phase 2 (fundamental wave)'},
    152: {'unit': 'VA', 'short_desc': 'S3_f', 'long_desc': 'Apparent power in phase 3 (fundamental wave)'},
    
    153: {'unit': '-', 'short_desc': 'PF1_h', 'long_desc': 'Power factor phase 1 including harmonics'},
    154: {'unit': '-', 'short_desc': 'PF2_h', 'long_desc': 'Power factor phase 2 including harmonics'},
    155: {'unit': '-', 'short_desc': 'PF3_h', 'long_desc': 'Power factor phase 3 including harmonics'},

    156: {'unit': '-', 'short_desc': 'LoadType1_f', 'long_desc': 'Load type indicator phase 1 (fundamental)'},
    157: {'unit': '-', 'short_desc': 'LoadType2_f', 'long_desc': 'Load type indicator phase 2 (fundamental)'},
    158: {'unit': '-', 'short_desc': 'LoadType3_f', 'long_desc': 'Load type indicator phase 3 (fundamental)'},

    159: {'unit': 'A', 'short_desc': 'I_sys_f', 'long_desc': 'System current of the fundamental wave'},
    160: {'unit': 'V', 'short_desc': 'U_line_avg_f', 'long_desc': 'Average line voltage (fundamental wave)'},
    161: {'unit': 'V', 'short_desc': 'U_phase_avg_f', 'long_desc': 'Average phase voltage (fundamental wave)'},
    # 162: {'unit': 'var', 'short_desc': 'Q_total_arith_f', 'long_desc': 'Arithmetic sum of reactive power (fundamental)'},
    # 163: {'unit': 'VA', 'short_desc': 'S_total_arith_f', 'long_desc': 'Arithmetic sum of apparent power (fundamental)'},
    # 164: {'unit': '-', 'short_desc': 'PF_total_arith_f', 'long_desc': 'Arithmetic power factor (fundamental wave)'},

    # 165: {'unit': '°', 'short_desc': 'Angle_UI1_f', 'long_desc': 'Phase angle between voltage and current in phase 1 (fundamental)'},
    # 166: {'unit': '°', 'short_desc': 'Angle_UI2_f', 'long_desc': 'Phase angle between voltage and current in phase 2 (fundamental)'},
    # 167: {'unit': '°', 'short_desc': 'Angle_UI3_f', 'long_desc': 'Phase angle between voltage and current in phase 3 (fundamental)'},

    # 168: {'unit': '-', 'short_desc': 'cos_phi1_f', 'long_desc': 'Cosine of phase angle in phase 1 (fundamental)'},
    # 169: {'unit': '-', 'short_desc': 'cos_phi2_f', 'long_desc': 'Cosine of phase angle in phase 2 (fundamental)'},
    # 170: {'unit': '-', 'short_desc': 'cos_phi3_f', 'long_desc': 'Cosine of phase angle in phase 3 (fundamental)'},
    
    171: {'unit': '°', 'short_desc': 'Angle_U1_f', 'long_desc': 'Voltage angle of phase U1 (fundamental)'},
    172: {'unit': '°', 'short_desc': 'Angle_U2_f', 'long_desc': 'Voltage angle of phase U2 (fundamental)'},
    173: {'unit': '°', 'short_desc': 'Angle_U3_f', 'long_desc': 'Voltage angle of phase U3 (fundamental)'},

    174: {'unit': '-', 'short_desc': 'RotField_f', 'long_desc': 'Rotating field direction of fundamental wave'},
}

PV_CHANNELS_MEASUREMENT_DICT = {
    2: {'unit': 'Wh', 'short_desc': 'DailyYield', 'long_desc': 'Energy yield produced today (PV)'},
    # 3: {'unit': 'A', 'short_desc': 'DC_Current_1', 'long_desc': 'DC current at input 1'},
    # 4: {'unit': 'V', 'short_desc': 'DC_Voltage_1', 'long_desc': 'DC voltage at input 1'},
    # 5: {'unit': 'W', 'short_desc': 'DC_Power_1', 'long_desc': 'DC power at input 1'},
    6: {'unit': 'W', 'short_desc': 'AC_ActivePower', 'long_desc': 'AC active power output of the inverter (PV)'},
    # 8: {'unit': 'A', 'short_desc': 'DC_Current_2', 'long_desc': 'DC current at input 2'},
    # 9: {'unit': 'V', 'short_desc': 'DC_Voltage_2', 'long_desc': 'DC voltage at input 2'},
    # 10: {'unit': 'W', 'short_desc': 'DC_Power_2', 'long_desc': 'DC power at input 2'}
}

In [30]:
from __future__ import annotations
from pathlib import Path
from typing import Dict, Tuple, List, Optional
import pandas as pd

# 2.) Define Measurement Limits
import pandas as pd
from math import sqrt
from collections.abc import Iterable

def build_limits_dfs(
    machine_specs: dict,
    *,
    same_limits_dict: dict | None = None,
    same_limits_items: list[tuple[tuple[float,float], Iterable[str]]] | None = None,
    safety_margin: float = 1.1,
    min_val: float = 0.0,
    vll_3p: float = 400.0,
    vll_1p: float = 230.0,
    print_info: bool = True,
) -> tuple[pd.DataFrame, pd.DataFrame | None]:
    """
    Returns:
      limits_df : DataFrame (index=machines, columns=measurements, values=(min,max))
      global_df : DataFrame (index=['ALL_MACHINES'], columns=ALL global measurements, values=(min,max))
                  or None if no global limits were provided.

    Pass *either*:
      - same_limits_items: [ ((min,max), [name,...]), ... ]   # preserves duplicates
      - same_limits_dict : { (min,max): [name,...], ... }     # duplicates in a dict literal are overwritten!
    """
    rt3 = sqrt(3)

    # ---- per-machine DF column layout ----
    CUR_PER_PHASE = ("I1","I1_f","I2","I2_f","I3","I3_f","IN","IN_f")
    CUR_SYSTEM    = ("I_sys","I_sys_f")

    def _phase_cols(prefix: str):
        return (f"{prefix}1", f"{prefix}1_f",
                f"{prefix}2", f"{prefix}2_f",
                f"{prefix}3", f"{prefix}3_f")

    POW_PHASE_S = _phase_cols("S")
    POW_PHASE_P = _phase_cols("P")
    POW_PHASE_Q = _phase_cols("Q")
    POW_TOTAL   = ("S_total","S_total_f","P_total","P_total_f","Q_total","Q_total_f")

    columns = [*CUR_PER_PHASE, *CUR_SYSTEM, *POW_PHASE_S, *POW_PHASE_P, *POW_PHASE_Q, *POW_TOTAL]

    # ---- per-machine DF ----
    limits_df = pd.DataFrame(index=list(machine_specs.keys()), columns=columns, dtype=object)
    for machine, (ph, I) in machine_specs.items():
        is3p = str(ph).upper().startswith("3P")
        I = float(I)

        cap_Ip = round(safety_margin * I, 6)
        cap_Is = round(safety_margin * (rt3 * I if is3p else I), 6)

        if is3p:
            cap_total = round(safety_margin * rt3 * vll_3p * I, 6)
            cap_phase = round(safety_margin * (vll_3p / rt3) * I, 6)
        else:
            cap_total = round(safety_margin * vll_1p * I, 6)
            cap_phase = cap_total

        for c in CUR_PER_PHASE: limits_df.at[machine, c] = (min_val, cap_Ip)
        for c in CUR_SYSTEM:    limits_df.at[machine, c] = (min_val, cap_Is)

        # Power per-phase:
        #   S*  -> [0, +max]
        #   P*, Q* -> [-max, +max]
        for c in POW_PHASE_S:
            limits_df.at[machine, c] = (0.0, cap_phase)
        for c in (*POW_PHASE_P, *POW_PHASE_Q):
            limits_df.at[machine, c] = (-cap_phase, cap_phase)

        # Power totals:
        #   S_total*  -> [0, +max]
        #   P_total*, Q_total* -> [-max, +max]
        for c in ("S_total", "S_total_f"):
            limits_df.at[machine, c] = (0.0, cap_total)
        for c in ("P_total", "P_total_f", "Q_total", "Q_total_f"):
            limits_df.at[machine, c] = (-cap_total, cap_total)

    # ---- global DF (flatten with FULL preservation) ----
    global_df = None
    flat_pairs: list[tuple[str, tuple[float,float]]] = []

    if same_limits_items:  # preferred: preserves duplicates by design
        for (mn, mx), names in same_limits_items:
            lim = (float(mn), float(mx))
            for n in names:
                flat_pairs.append((str(n), lim))
    elif same_limits_dict:  # OK if keys are unique; duplicates already overwritten upstream
        for key, names in same_limits_dict.items():
            if not (isinstance(key, tuple) and len(key) == 2):
                raise TypeError("same_limits_dict must have (min,max) tuple keys.")
            lim = (float(key[0]), float(key[1]))
            for n in names:
                flat_pairs.append((str(n), lim))

    if flat_pairs:
        cols = [n for n, _ in flat_pairs]  # keep order; allow duplicates, last wins for same name
        global_df = pd.DataFrame(index=["ALL_MACHINES"], columns=cols, dtype=object)
        for n, lim in flat_pairs:
            global_df.at["ALL_MACHINES", n] = lim

    if print_info:
        print(
            "Limits calculation:\n"
            f"  safety_margin M = {safety_margin}, min_val = {min_val}\n"
            f"  voltages: V_LL_3P = {vll_3p} V, V_1P = {vll_1p} V\n"
            "  Current per-phase (I1,I2,I3,IN,*_f):   M * I\n"
            "  Current system (I_sys,*_f):           3P -> M*√3*I   |  1P -> M*I\n"
            "  Power totals (S/P/Q _total,*_f):      3P -> M*√3*V_LL_3P*I   |  1P -> M*V_1P*I\n"
            "  Power per-phase (S/P/Q 1..3,*_f):     3P -> M*(V_LL_3P/√3)*I |  1P -> M*V_1P*I\n"
            "Note: Use same_limits_items=[((min,max), [..]), ...] to preserve duplicate (min,max) groups."
        )

    return limits_df, global_df


# Your compact machine specs (no voltages here):
MACHINE_SPECS = {
    "EPI_ChipPress": ("3P", 50.0),
    "EPI_ChipSaw": ("3P", 6.0),
    "EPI_HighTempOven": ("3P", 32.0),
    "EPI_PickAndPlace": ("1P", 5.0),
    "EPI_PumpStation1": ("3P", 34.0),
    "EPI_PumpStation2": ("3P", 36.0),
    "EPI_ScreenPrinter": ("1P", 5.2),
    "EPI_SolderOven": ("3P", 29.0),
    "EPI_TotalLoad": ("3P", 160.0),
    "EPI_VacuumSoldering": ("3P", 14.0),
    "EPI_WashingMachine": ("3P", 18.0),
    "TEC_48S": ("3P", 40.0),
    "TEC_CFST161": ("3P", 40.0),
    "TEC_CTX800TC": ("3P", 100.0),
    "TEC_Chiron800": ("3P", 90.0),
    "TEC_DMF3008": ("3P", 80.0),
    "TEC_DMU125MB": ("3P", 125.0),
    "TEC_DNG50evo": ("3P", 80.0),
    "TEC_E110": ("3P", 80.0),
    "TEC_E30D2": ("3P", 15.0),
    "TEC_JWA24": ("3P", 45.0),
    "TEC_MV2400R": ("3P", 70.0),
}

# Example: dictionary of global same-for-all limits (replace with yours)
# Preserve both (0.0,100.0) groups by listing them separately:
SAME_LIMITS_ITEMS = [
    ((49.0, 51.0), ["Freq", "Freq_f"]),
    ((-1.0, 1.0), ["LoadType1","LoadType1_f","LoadType2","LoadType2_f","LoadType3","LoadType3_f"]),
    ((-180.0, 180.0), ["Angle_U1","Angle_U1_f","Angle_U2","Angle_U2_f","Angle_U3","Angle_U3_f"]),
    ((2000.0, 25000.0), ["OpHours"]),
    # --- (0.0,100.0) VOLTAGE harmonics ---
    ((0.0, 100.0), [
        "U12_h2","U12_h3","U12_h4","U12_h5","U12_h7","U12_h9","U12_h11","U12_h13","U12_h15","U12_h17","U12_h19","U12_h21","U12_h23","U12_h25","U12_h27","U12_h29","U12_h31",
        "U1_h2","U1_h3","U1_h4","U1_h5","U1_h7","U1_h9","U1_h11","U1_h13","U1_h15","U1_h17","U1_h19","U1_h21","U1_h23","U1_h25","U1_h27","U1_h29","U1_h31",
        "U23_h2","U23_h3","U23_h4","U23_h5","U23_h7","U23_h9","U23_h11","U23_h13","U23_h15","U23_h17","U23_h19","U23_h21","U23_h23","U23_h25","U23_h27","U23_h29","U23_h31",
        "U2_h2","U2_h3","U2_h4","U2_h5","U2_h7","U2_h9","U2_h11","U2_h13","U2_h15","U2_h17","U2_h19","U2_h21","U2_h23","U2_h25","U2_h27","U2_h29","U2_h31",
        "U31_h2","U31_h3","U31_h4","U31_h5","U31_h7","U31_h9","U31_h11","U31_h13","U31_h15","U31_h17","U31_h19","U31_h21","U31_h23","U31_h25","U31_h27","U31_h29","U31_h31",
        "U3_h2","U3_h3","U3_h4","U3_h5","U3_h7","U3_h9","U3_h11","U3_h13","U3_h15","U3_h17","U3_h19","U3_h21","U3_h23","U3_h25","U3_h27","U3_h29","U3_h31",
    ]),
    # --- (0.0,100.0) CURRENT harmonics ---
    ((0.0, 100.0), [
        "I1_h2","I1_h3","I1_h4","I1_h5","I1_h7","I1_h9","I1_h11","I1_h13","I1_h15","I1_h17","I1_h19","I1_h21","I1_h23","I1_h25","I1_h27","I1_h29","I1_h31",
        "I2_h2","I2_h3","I2_h4","I2_h5","I2_h7","I2_h9","I2_h11","I2_h13","I2_h15","I2_h17","I2_h19","I2_h21","I2_h23","I2_h25","I2_h27","I2_h29","I2_h31",
        "I3_h2","I3_h3","I3_h4","I3_h5","I3_h7","I3_h9","I3_h11","I3_h13","I3_h15","I3_h17","I3_h19","I3_h21","I3_h23","I3_h25","I3_h27","I3_h29","I3_h31",
        "IN_h2","IN_h3","IN_h4","IN_h5","IN_h7","IN_h9","IN_h11","IN_h13","IN_h15","IN_h17","IN_h19","IN_h21","IN_h23","IN_h25","IN_h27","IN_h29","IN_h31",
    ]),
    ((200.9, 260.0), ['U1','U1_f','U2','U2_f','U3','U3_f','U_phase_avg','U_phase_avg_f']),
    ((350.0, 450.0), ['U12','U12_f','U23','U23_f','U31','U31_f','U_line_avg','U_line_avg_f']),
    ((-1.0, 1.0), ['RotField','RotField_f']),
    ((-1.0, 1.0), ['PF1','PF1_h','PF2','PF2_h','PF3','PF3_h','PF_total','PF_total_f']),
    ((0.0, 11000), ['AC_ActivePower']),
    ((0.0, 50000), ['DailyYield']),
    ((0.0, 100), ['THD_U1','THD_U12','THD_U2','THD_U23','THD_U3','THD_U31']),
    ((0.0, 100), ['THD_I1','THD_I2','THD_I3','THD_IN']),
]

# Build both DFs (no values are lost)
limits_df, global_df = build_limits_dfs(
    MACHINE_SPECS,
    same_limits_items=SAME_LIMITS_ITEMS,   # << use the list form
    safety_margin=1.2,
    min_val=0.0,
    vll_3p=400.0,
    vll_1p=230.0,
    print_info=False,
)

# Optional rounded view for display
rounded_limits = limits_df.map(lambda t: (round(t[0], 1), round(t[1], 1)))
rounded_global = global_df.map(lambda t: (round(t[0], 1), round(t[1], 1)))

def build_catalog_and_check_limits(
    dataset_clean_dir: Path | str,
    validation_results_dir: Path | str,
    per_machine_df: pd.DataFrame,
    global_df: Optional[pd.DataFrame] = None,
    save_catalog: bool = False,
    out_path: Path | str = "../dataset_clean/00meta_data/measurement_catalog.json",
    head_n: int = 20,
) -> dict:
    """
    One-shot function that:
      1) Collects the unique measurements from dataset_clean and validation_results
         (treating folders/files ending with '_EMPTY' as "no values" and excluding them).
      2) Builds a measurement catalog DataFrame using the pasted config dicts:
           - EPI_CHANNELS_3_MEASUREMENT_DICT (highest precedence)
           - TH_CHANNELS_MEASUREMENT_DICT
           - EPI_CHANNELS_1_MEASUREMENT_DICT
           - PV_CHANNELS_MEASUREMENT_DICT   (lowest precedence)
         Only the discovered measurements are included.
      3) Checks coverage of limits (columns) in per_machine_df ∪ global_df versus the discovered measurements.

    Parameters
    ----------
    dataset_clean_dir : Path | str
        Root of 'dataset_clean' (expects structure: <MACHINE>/<MEASUREMENT or MEASUREMENT_EMPTY>/... .csv.xz).
    validation_results_dir : Path | str
        Root of 'validation_results' (expects: <MACHINE>/<MEASUREMENT>/<YEAR>_<MEASUREMENT>_stats.csv).
    per_machine_df : pd.DataFrame
        Per-machine limits DataFrame (columns = measurements; values are (min,max) tuples or NaN).
    global_df : Optional[pd.DataFrame]
        Global limits DataFrame (single row by contract; columns = measurements; values are (min,max) tuples).
    save_catalog : bool
        If True, writes the catalog JSON to out_path.
    out_path : Path | str
        Output path for the catalog JSON if save_catalog=True.
    head_n : int
        Number of head rows to print for a quick glance.

    Returns
    -------
    dict
        {
          "catalog_df": DataFrame,                 # ['measurement','unit','description']
          "coverage_report": dict,                 # see below
          "measurements": List[str],               # sorted unique measurements used
        }

        coverage_report contains:
          - total_expected
          - total_in_dfs
          - covered_count
          - missing_count
          - extras_count
          - overlap_count
          - different_overlap_count
          - missing: list[str]
          - extras: list[str]
          - present: list[str]
          - overlap_columns: list[str]
          - different_overlap: list[str]
    """
    dataset_clean_dir = Path(dataset_clean_dir)
    validation_results_dir = Path(validation_results_dir)

    # --------- 1) Discover unique measurements  ---------
    def _unique_measurements_from_dataset_clean(base_dir: Path) -> List[str]:
        """
        dataset_clean/<MACHINE>/<MEASUREMENT or MEASUREMENT_EMPTY>/<YEAR>_<MEASUREMENT or MEASUREMENT_EMPTY>.csv.xz
        Exclude folders ending with '_EMPTY'.
        Include a measurement if at least one non-EMPTY file exists.
        """
        measurements = set()
        if not base_dir.exists():
            return []
        for machine_dir in base_dir.iterdir():
            if not machine_dir.is_dir():
                continue
            for meas_dir in machine_dir.iterdir():
                if not meas_dir.is_dir():
                    continue
                meas_name_folder = meas_dir.name
                if meas_name_folder.endswith("_EMPTY"):
                    continue
                # Require at least one non-EMPTY file inside this folder
                has_non_empty = any("_EMPTY" not in fp.stem for fp in meas_dir.glob("*.csv.xz"))
                if has_non_empty:
                    measurements.add(meas_name_folder)
        return sorted(measurements)

    def _unique_measurements_from_validation_results(base_dir: Path) -> List[str]:
        """
        validation_results/<MACHINE>/<MEASUREMENT>/<YEAR>_<MEASUREMENT>_stats.csv
        """
        measurements = set()
        if not base_dir.exists():
            return []
        for fp in base_dir.rglob("*_stats.csv"):
            try:
                rel = fp.relative_to(base_dir)
            except Exception:
                continue
            parts = rel.parts
            if len(parts) < 3:
                continue
            measurement = parts[1]  # <MEASUREMENT>
            # If validation exported *_EMPTY stats (uncommon), still treat it as the base name:
            if measurement.endswith("_EMPTY"):
                measurement = measurement[:-6]
            measurements.add(measurement)
        return sorted(measurements)

    meas_dc = set(_unique_measurements_from_dataset_clean(dataset_clean_dir))
    meas_vr = set(_unique_measurements_from_validation_results(validation_results_dir))
    measurements = sorted(meas_dc | meas_vr)

    print("=== Measurement Discovery ===")
    print(f"From dataset_clean: {len(meas_dc)}")
    print(f"From validation_results: {len(meas_vr)}")
    print(f"Union (expected measurements): {len(measurements)}")

    # --------- 2) Build catalog using pasted dicts (first-wins)  ---------
    def _normalize(measurement_dict: Dict) -> Dict[str, Tuple[str, str]]:
        """
        Accepts:
          {"id": {"short_desc": "...", "unit": "...", "long_desc": "..."}, ...}
        Returns:
          {short_desc: (unit, long_desc)}
        """
        out: Dict[str, Tuple[str, str]] = {}
        if not isinstance(measurement_dict, dict):
            return out
        for entry in measurement_dict.values():
            if not isinstance(entry, dict):
                continue
            short = entry.get("short_desc")
            unit = entry.get("unit")
            long_desc = entry.get("long_desc")
            if short and (unit is not None) and (long_desc is not None):
                out[str(short)] = (str(unit), str(long_desc))
        return out

    def _merge_first_wins(maps_in_order: List[Dict[str, Tuple[str, str]]]) -> Dict[str, Tuple[str, str]]:
        merged: Dict[str, Tuple[str, str]] = {}
        for mp in maps_in_order:
            for k, v in mp.items():
                if k not in merged:
                    merged[k] = v
        return merged

    # Pull config dicts from globals if present (user pastes them above this function).
    cfg_maps = [
        _normalize(globals().get("EPI_CHANNELS_3_MEASUREMENT_DICT", {})),
        _normalize(globals().get("TH_CHANNELS_MEASUREMENT_DICT", {})),
        _normalize(globals().get("EPI_CHANNELS_1_MEASUREMENT_DICT", {})),
        _normalize(globals().get("PV_CHANNELS_MEASUREMENT_DICT", {})),
    ]
    config_map = _merge_first_wins(cfg_maps)

    # Build catalog strictly for the discovered measurements
    rows = []
    for m in measurements:
        unit, desc = config_map.get(m, ("", ""))
        rows.append({"measurement": m, "unit": unit, "description": desc})
    catalog_df = (
        pd.DataFrame(rows, columns=["measurement", "unit", "description"])
        .sort_values("measurement")
        .reset_index(drop=True)
    )

    if save_catalog:
        out_path = Path(out_path)
        out_path.parent.mkdir(parents=True, exist_ok=True)
        catalog_df.to_json(out_path, orient="records", force_ascii=False, indent=2)
        print(f"[✓] Saved measurement catalog (JSON) → {out_path}  ({catalog_df.shape[0]} rows)")
    else:
        print(f"Measurement catalog shape: {catalog_df.shape}")
        print(catalog_df.head(head_n).to_string(index=False))

    # --------- 3) Limits coverage check against discovered measurements  ---------
    def _check_limits_coverage(
        expected_measurements: List[str],
        per_machine_df: pd.DataFrame,
        global_df: Optional[pd.DataFrame],
    ) -> dict:
        expected_set = set(expected_measurements)
        per_cols = set(per_machine_df.columns)
        glob_cols = set(global_df.columns) if (global_df is not None) else set()
        all_cols = per_cols | glob_cols

        missing = sorted(expected_set - all_cols)
        extras = sorted(all_cols - expected_set)
        present = sorted(expected_set & all_cols)

        overlap_columns = sorted(per_cols & glob_cols)

        different_overlap = []
        if global_df is not None and not global_df.empty and overlap_columns:
            # Assume global_df has a single row by contract ("ALL_MACHINES")
            grow = global_df.iloc[0]
            for col in overlap_columns:
                gval = grow[col]
                pm_vals = per_machine_df[col].dropna().unique()
                if any(v != gval for v in pm_vals):
                    different_overlap.append(col)

        report = {
            "total_expected": len(expected_set),
            "total_in_dfs": len(all_cols),
            "covered_count": len(present),
            "missing_count": len(missing),
            "extras_count": len(extras),
            "overlap_count": len(overlap_columns),
            "different_overlap_count": len(different_overlap),
            "missing": missing,
            "extras": extras,
            "present": present,
            "overlap_columns": overlap_columns,
            "different_overlap": sorted(different_overlap),
        }

        # Pretty print
        print("\n=== Limits Coverage Check ===")
        print(f"Expected measurements: {report['total_expected']}")
        print(f"Columns in DFs union: {report['total_in_dfs']}")
        print(f"Covered (expected ∩ DFs): {report['covered_count']}")
        print(f"Missing (expected \\ DFs): {report['missing_count']}")
        print(f"Extras (DFs \\ expected): {report['extras_count']}")
        print(f"Overlap cols (per-machine ∩ global): {report['overlap_count']}")
        print(f"Overlap with differing tuples: {report['different_overlap_count']}")
        if report["missing"]:
            print("\n— Missing —")
            print(report["missing"])
        if report["extras"]:
            print("\n— Extras —")
            print(report["extras"])
        if report["different_overlap"]:
            print("\n— Overlap with different tuples —")
            print(report["different_overlap"])

        return report

    coverage_report = _check_limits_coverage(measurements, per_machine_df, global_df)

    return {
        "catalog_df": catalog_df,
        "coverage_report": coverage_report,
        "measurements": measurements,
    }

# ---------------------------
# Example (comment out in production):
result = build_catalog_and_check_limits(
    dataset_clean_dir="../dataset_clean",
    validation_results_dir="../dataset_clean_validation",
    per_machine_df=rounded_limits,      # your per-machine DF
    global_df=rounded_global,           # your global DF (or None)
    save_catalog=False,
    out_path="../dataset_clean/00meta_data/00measurement_deatils.json",
    head_n=20,
)


=== Measurement Discovery ===
From dataset_clean: 256
From validation_results: 257
Union (expected measurements): 257
[✓] Saved measurement catalog (JSON) → ..\dataset_clean\00meta_data\measurement_catalog.json  (257 rows)

=== Limits Coverage Check ===
Expected measurements: 257
Columns in DFs union: 257
Covered (expected ∩ DFs): 257
Missing (expected \ DFs): 0
Extras (DFs \ expected): 0
Overlap cols (per-machine ∩ global): 0
Overlap with differing tuples: 0


In [33]:
from __future__ import annotations
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple
import json
import pandas as pd
import re

def build_exogenous_data_details(
    exogenous_dir: str | Path = "dataset_clean/01exogenous_data",
    out_path: str | Path = "dataset_clean/00meta_data/00exogenous_data_details.json",
    preview_print: bool = True,
) -> Dict[str, Any]:
    """
    Create 00exogenous_data_details.json describing each exogenous dataset:
      - description (what the file contains)
      - file, frequency, time_column, timezone, timerange{start,end}, row_count
      - columns[]: {name, unit, description}

    The function is tailored to the following files (if present):
      - electricity_emissions_15min_transnetbw_2018_2024.csv
      - electricity_prices_15min_de_2018_2024.csv
      - holidays_kalsruhe_bw_2018_2024.csv
      - weather_data_karlsruhe_hourly_2018_2024.csv
      - weather_data_stations_used.csv
    """
    exogenous_dir = Path(exogenous_dir)
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    # ----- Units & descriptions knowledge base -----
    UNIT_DESC_MAP: Dict[str, Dict[str, str]] = {
        # time columns
        "utc_datetime": {"unit": "", "desc": "Coordinated Universal Time timestamp."},
        "date": {"unit": "", "desc": "Calendar date (local)."},
        # emissions mix
        "nuclear": {"unit": "MW", "desc": "Nuclear generation (regional mix)."},
        "hydro": {"unit": "MW", "desc": "Hydropower generation."},
        "wind_onshore": {"unit": "MW", "desc": "Onshore wind generation."},
        "solar_pv": {"unit": "MW", "desc": "Solar PV generation."},
        "biomass": {"unit": "MW", "desc": "Biomass-based generation."},
        "other_renewables": {"unit": "MW", "desc": "Other renewable generation (e.g., waste, geothermal)."},
        "natural_gas": {"unit": "MW", "desc": "Gas-fired generation."},
        "hard_coal": {"unit": "MW", "desc": "Hard coal generation."},
        "other_conventional": {"unit": "MW", "desc": "Other conventional generation (e.g., oil, others)."},
        "pumped_storage_generation": {"unit": "MW", "desc": "Pumped storage generation (turbining)."},
        "emission_intensity_g_per_kwh": {"unit": "g/kWh", "desc": "Grid emission intensity."},
        # prices
        "price_eur_per_mwh": {"unit": "EUR/MWh", "desc": "Day-ahead wholesale electricity price."},
        # holidays
        "weekday": {"unit": "", "desc": "Day of week (string)."},
        "is_holiday": {"unit": "", "desc": "Holiday indicator (boolean)."},
        "holiday_name": {"unit": "", "desc": "Holiday name (empty if not a holiday)."},
        # weather
        "temperature_air_mean_2m_°C": {"unit": "°C", "desc": "Mean air temperature at 2 m."},
        "temperature_dew_point_mean_2m_°C": {"unit": "°C", "desc": "Mean dew-point temperature at 2 m."},
        "humidity_%": {"unit": "%", "desc": "Relative humidity."},
        "wind_speed_m/s": {"unit": "m/s", "desc": "Wind speed."},
        "wind_direction_°": {"unit": "°", "desc": "Wind direction (meteorological)."},
        "wind_gust_max_m/s": {"unit": "m/s", "desc": "Maximum wind gust within the hour."},
        "pressure_air_site_hPa": {"unit": "hPa", "desc": "Station pressure."},
        "pressure_air_sea_level_hPa": {"unit": "hPa", "desc": "Sea-level pressure (reduced)."},
        "radiation_global_J/cm²_per_hour": {"unit": "J/cm² per hour", "desc": "Global shortwave radiation per hour."},
        "sunshine_duration_minutes_per_hour": {"unit": "min/h", "desc": "Sunshine duration per hour."},
        "cloud_cover_total_oktas_(0–8)": {"unit": "oktas", "desc": "Total cloud cover (0–8)."},
        "visibility_range_m": {"unit": "m", "desc": "Visibility range."},
        "precipitation_height_mm_per_hour": {"unit": "mm/h", "desc": "Precipitation height per hour."},
        # stations table
        "parameter": {"unit": "", "desc": "Meteorological parameter key."},
        "station_id": {"unit": "", "desc": "DWD station identifier."},
        "name": {"unit": "", "desc": "Station name."},
        "state": {"unit": "", "desc": "Federal state."},
        "distance": {"unit": "km", "desc": "Distance to reference location (if provided)."},
        "height": {"unit": "m", "desc": "Station elevation."},
        "latitude": {"unit": "°", "desc": "Latitude (WGS84)."},
        "longitude": {"unit": "°", "desc": "Longitude (WGS84)."},
    }

    UNIT_PATTERNS: List[Tuple[re.Pattern, str]] = [
        (re.compile(r"_eur_per_mwh$"), "EUR/MWh"),
        (re.compile(r"_g_per_kwh$"), "g/kWh"),
        (re.compile(r"_m/s$"), "m/s"),
        (re.compile(r"_hPa$"), "hPa"),
        (re.compile(r"_mm_per_hour$"), "mm/h"),
        (re.compile(r"_minutes_per_hour$"), "min/h"),
        (re.compile(r"_oktas"), "oktas"),
        (re.compile(r"_°$"), "°"),
        (re.compile(r"_°C$"), "°C"),
        (re.compile(r"_J/cm²_per_hour$"), "J/cm² per hour"),
        (re.compile(r"_m$"), "m"),
        (re.compile(r"_%$"), "%"),
    ]

    def infer_unit(col: str) -> str:
        if col in UNIT_DESC_MAP:
            return UNIT_DESC_MAP[col]["unit"]
        for pat, unit in UNIT_PATTERNS:
            if pat.search(col):
                return unit
        suf = col.split("_")[-1]
        if suf in {"m", "km", "hPa", "mm", "°C", "°", "%"}:
            return suf
        return ""

    def infer_desc(col: str) -> str:
        if col in UNIT_DESC_MAP:
            return UNIT_DESC_MAP[col]["desc"]
        txt = col.replace("_", " ").strip()
        return f"{txt[0].upper()}{txt[1:]}." if txt else ""

    def detect_freq(name: str) -> str:
        n = name.lower()
        if "15min" in n or "15_min" in n or "quarter" in n:
            return "15min"
        if "hour" in n:
            return "hourly"
        if "holiday" in n or "daily" in n or "day" in n:
            return "daily"
        return ""

    def read_time_bounds(fp: Path, time_col: str, parse_utc: bool) -> tuple[Optional[pd.Timestamp], Optional[pd.Timestamp], int]:
        """Get first/last timestamp and total row count efficiently."""
        # start
        start = None
        try:
            head = pd.read_csv(fp, usecols=[time_col], nrows=1)
            if parse_utc:
                head[time_col] = pd.to_datetime(head[time_col], utc=True, errors="coerce")
            start = head[time_col].dropna().iloc[0] if not head.empty else None
        except Exception:
            pass
        # end + count (chunked)
        last = None
        total = 0
        try:
            for chunk in pd.read_csv(fp, usecols=[time_col], chunksize=200_000):
                total += len(chunk)
                if parse_utc:
                    chunk[time_col] = pd.to_datetime(chunk[time_col], utc=True, errors="coerce")
                non_null = chunk[time_col].dropna()
                if not non_null.empty:
                    last = non_null.iloc[-1]
        except Exception:
            pass
        return start, last, total

    def columns_with_meta(fp: Path) -> List[Dict[str, str]]:
        """Robust header extraction with units/descriptions."""
        try:
            cols = pd.read_csv(fp, nrows=0).columns.tolist()
        except Exception:
            # fallback: manual first line
            try:
                with open(fp, "r", encoding="utf-8") as f:
                    cols = f.readline().strip().split(",")
            except Exception:
                cols = []
        # fix known typo in stations file: "arameter" -> "parameter"
        cols = [("parameter" if c == "arameter" else c) for c in cols]
        return [{"name": c, "unit": infer_unit(c), "description": infer_desc(c)} for c in cols]

    def describe_file(filename: str) -> str:
        fn = filename.lower()
        if fn.startswith("electricity_emissions"):
            return ("Quarter-hourly regional generation mix (MW) by technology and corresponding grid "
                    "emission intensity (g/kWh) for the TransnetBW area (UTC).")
        if fn.startswith("electricity_prices"):
            return ("Quarter-hourly German wholesale electricity price time series (UTC), values in EUR/MWh; "
                    "missing entries indicate unavailable market data for the interval.")
        if fn.startswith("holidays_kalsruhe"):
            return ("Daily calendar of public holidays for Karlsruhe (Baden-Württemberg), including weekday, "
                    "boolean holiday flag, and holiday name (local time).")
        if fn.startswith("weather_data_karlsruhe_hourly"):
            return ("Hourly meteorological observations for Karlsruhe (UTC): temperature, humidity, wind, "
                    "pressure, radiation, sunshine duration, cloud cover, visibility, and precipitation.")
        if fn.startswith("weather_data_stations_used"):
            return ("Lookup table of DWD stations used per parameter, including station ID, name, location, "
                    "distance to reference, and elevation.")
        return "Exogenous dataset."

    datasets: List[Dict[str, Any]] = []

    # Emitters: define expected files and their time/zone handling
    FILE_SPECS = [
        # (filename, time_column, parse_utc, explicit_freq, explicit_tz)
        ("electricity_emissions_15min_transnetbw_2018_2024.csv", "utc_datetime", True, "15min", "UTC"),
        ("electricity_prices_15min_de_2018_2024.csv",            "utc_datetime", True, "15min", "UTC"),
        ("holidays_kalsruhe_bw_2018_2024.csv",                   "date",         False, "daily", "local"),
        ("weather_data_karlsruhe_hourly_2018_2024.csv",          "utc_datetime", True, "hourly", "UTC"),
        ("weather_data_stations_used.csv",                       "",             False, "static_table", ""),
    ]

    for fname, tcol, parse_utc, freq, tz in FILE_SPECS:
        fpath = exogenous_dir / fname
        if not fpath.exists():
            continue

        cols_meta = columns_with_meta(fpath)

        # time bounds + row count when a time column exists
        if tcol:
            start, end, nrows = read_time_bounds(fpath, tcol, parse_utc=parse_utc)
            tr = {
                "start": None if start is None else start.isoformat() if parse_utc else pd.to_datetime(start).date().isoformat(),
                "end":   None if end   is None else end.isoformat()   if parse_utc else pd.to_datetime(end).date().isoformat(),
            }
        else:
            nrows = max(sum(1 for _ in open(fpath, "r", encoding="utf-8")) - 1, 0)
            tr = {"start": None, "end": None}

        datasets.append({
            "description": describe_file(fname),
            "file": fpath.name,
            "frequency": freq or detect_freq(fpath.name),
            "time_column": tcol,
            "timezone": tz,
            "timerange": tr,
            "row_count": nrows,
            "columns": cols_meta,   # now correctly populated
        })

    result = {"base_dir": str(exogenous_dir), "datasets": datasets}

    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

    if preview_print:
        print(f"[✓] Saved exogenous data details → {out_path}")
        print(f"Datasets found: {len(datasets)}")
        for d in datasets:
            print(f"  - {d['file']}  [{d['frequency']}] — {d['description']}")

    return result

# Example:
build_exogenous_data_details(
    exogenous_dir="../dataset_clean/01exogenous_data",
    out_path="../dataset_clean/00meta_data/00exogenous_data_details.json",
    preview_print=True,
)


[✓] Saved exogenous data details → ..\dataset_clean\00meta_data\00exogenous_data_details.json
Datasets found: 5
  - electricity_emissions_15min_transnetbw_2018_2024.csv  in] — Quarter-hourly regional generation mix (MW) by technology and corresponding grid emission intensity (g/kWh) for the TransnetBW area (UTC).
  - electricity_prices_15min_de_2018_2024.csv  in] — Quarter-hourly German wholesale electricity price time series (UTC), values in EUR/MWh; missing entries indicate unavailable market data for the interval.
  - holidays_kalsruhe_bw_2018_2024.csv  [daily] — Daily calendar of public holidays for Karlsruhe (Baden-Württemberg), including weekday, boolean holiday flag, and holiday name (local time).
  - weather_data_karlsruhe_hourly_2018_2024.csv  [hourly] — Hourly meteorological observations for Karlsruhe (UTC): temperature, humidity, wind, pressure, radiation, sunshine duration, cloud cover, visibility, and precipitation.
  - weather_data_stations_used.csv  [static_table] — Look

{'base_dir': '..\\dataset_clean\\01exogenous_data',
 'datasets': [{'description': 'Quarter-hourly regional generation mix (MW) by technology and corresponding grid emission intensity (g/kWh) for the TransnetBW area (UTC).',
   'file': 'electricity_emissions_15min_transnetbw_2018_2024.csv',
   'frequency': '15min',
   'time_column': 'utc_datetime',
   'timezone': 'UTC',
   'timerange': {'start': '2018-01-01T00:00:00+00:00',
    'end': '2024-12-31T23:45:00+00:00'},
   'row_count': 245472,
   'columns': [{'name': 'utc_datetime',
     'unit': '',
     'description': 'Coordinated Universal Time timestamp.'},
    {'name': 'nuclear',
     'unit': 'MW',
     'description': 'Nuclear generation (regional mix).'},
    {'name': 'hydro', 'unit': 'MW', 'description': 'Hydropower generation.'},
    {'name': 'wind_onshore',
     'unit': 'MW',
     'description': 'Onshore wind generation.'},
    {'name': 'solar_pv', 'unit': 'MW', 'description': 'Solar PV generation.'},
    {'name': 'biomass',
     'uni

In [32]:
result

{'base_dir': '..\\dataset_clean\\01exogenous_data',
 'datasets': [{'logical_name': 'electricity_emissions_transnetbw',
   'file': 'electricity_emissions_15min_transnetbw_2018_2024.csv',
   'frequency': '15min',
   'time_column': 'utc_datetime',
   'timezone': 'UTC',
   'timerange': {'start': '2018-01-01T00:00:00+00:00',
    'end': '2024-12-31T23:45:00+00:00'},
   'row_count': 245472,
   'columns': []},
  {'logical_name': 'electricity_prices_de',
   'file': 'electricity_prices_15min_de_2018_2024.csv',
   'frequency': '15min',
   'time_column': 'utc_datetime',
   'timezone': 'UTC',
   'timerange': {'start': '2018-01-01T00:00:00+00:00',
    'end': '2024-12-31T23:45:00+00:00'},
   'row_count': 245472,
   'columns': []},
  {'logical_name': 'holidays_karlsruhe_bw',
   'file': 'holidays_kalsruhe_bw_2018_2024.csv',
   'frequency': 'daily',
   'time_column': 'date',
   'timezone': 'local',
   'timerange': {'start': '2018-01-01', 'end': '2024-12-31'},
   'row_count': 2557,
   'columns': []},
  {